# TPC-H ETL benchmark with Ray Data

**Time to complete**: 35 min | **Difficulty**: Intermediate | **Prerequisites**: Python, data processing experience

This template demonstrates how to build production-ready ETL pipelines using Ray Data for distributed data processing. You'll learn to process large-scale datasets efficiently while understanding the underlying architecture that makes Ray Data powerful for both traditional ETL and modern AI workloads.

## Table of Contents

1. [Prerequisites and Setup](#prerequisites-and-setup) (3 min)
2. [Quick Start: Your First ETL Pipeline](#quick-start-your-first-etl-pipeline) (5 min)
3. [Understanding Ray Data and ETL](#understanding-ray-data-and-etl) (4 min)
4. [Ray Data Architecture & Concepts](#ray-data-architecture--concepts) (5 min)
5. [Extract: Reading Data from Multiple Sources](#extract-reading-data-from-multiple-sources) (6 min)
6. [Transform: Processing and Cleaning Data](#transform-processing-and-cleaning-data) (8 min)
7. [Load: Writing Data to Destinations](#load-writing-data-to-destinations) (4 min)
8. [Advanced ETL Patterns and Optimizations](#advanced-etl-patterns-and-optimizations) (5 min)
9. [Performance Monitoring and Best Practices](#performance-monitoring-and-best-practices) (3 min)
10. [Troubleshooting and Production Considerations](#troubleshooting-and-production-considerations) (2 min)

## Learning Objectives

By completing this template, you will understand:

**Why ETL matters for modern data teams:**
- ETL is the foundation of data-driven decision making, enabling businesses to transform raw data into actionable insights
- Modern ETL must handle both structured business data and unstructured AI content at scale
- Traditional ETL tools struggle with Python-native workflows and multimodal data processing

**Ray Data's ETL superpowers:**
- **Unified Platform**: Process traditional ETL and AI workloads on the same infrastructure
- **Python-Native Performance**: No JVM overhead - pure Python performance at scale
- **Automatic Scaling**: Seamlessly scale from single machines to thousands of cores
- **Streaming Architecture**: Handle datasets larger than cluster memory with ease
- **Production Ready**: Built-in fault tolerance, monitoring, and enterprise features

**What you'll be able to do:**
- Build scalable ETL pipelines that process millions of records efficiently
- Optimize performance using Ray Data's advanced features like operator fusion
- Handle real-world data quality issues and implement robust error handling
- Deploy production-ready ETL pipelines with proper monitoring and observability
- Choose the right Ray Data operations for different ETL scenarios


## Prerequisites and Setup

### Prerequisites Checklist

Before starting this template, ensure you have:

- [ ] **Python 3.8+** installed on your system
- [ ] **Basic Python knowledge** including functions, classes, and data structures
- [ ] **Data processing experience** with pandas, numpy, or similar libraries
- [ ] **Understanding of ETL concepts** (Extract, Transform, Load)
- [ ] **Access to a Ray cluster** or ability to run Ray locally
- [ ] **8GB+ RAM** recommended for optimal performance
- [ ] **Internet connection** for downloading dependencies and data

### System Requirements

**Minimum Requirements:**
- CPU: 4 cores
- RAM: 8GB
- Storage: 2GB free space
- Python: 3.8+

**Recommended for Production:**
- CPU: 16+ cores
- RAM: 32GB+
- Storage: 10GB+ free space
- Python: 3.9+

### Installation and Verification

Let's verify your environment and install required dependencies:


In [ ]:
# Standard library imports
import os
import sys
import time
import logging
from typing import Dict, Any, List, Tuple, Union
from datetime import datetime, timedelta

# Third-party imports
import ray
import pandas as pd
import numpy as np
import pyarrow as pa
from ray.data import DataContext, Dataset
from ray.data.aggregate import Count, Mean, Sum, Min, Max

# Configure logging for better debugging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def verify_environment() -> bool:
    """Verify that the environment meets requirements."""
    try:
        # Check Python version
        python_version = sys.version_info
        if python_version.major < 3 or (python_version.major == 3 and python_version.minor < 8):
            logger.error(f"Python 3.8+ required, found {python_version.major}.{python_version.minor}")
            return False
        
        # Check available memory (approximate)
        import psutil
        memory_gb = psutil.virtual_memory().total / (1024**3)
        if memory_gb < 4:
            logger.warning(f"Low memory detected: {memory_gb:.1f}GB. 8GB+ recommended.")
        
        logger.info(f"Environment verification passed - Python {python_version.major}.{python_version.minor}, {memory_gb:.1f}GB RAM")
        return True
        
    except ImportError as e:
        logger.error(f"Missing required package: {e}")
        return False
    except Exception as e:
        logger.error(f"Environment verification failed: {e}")
        return False

# Verify environment before proceeding
if not verify_environment():
    raise RuntimeError("Environment verification failed. Please check prerequisites.")

# Configure Ray Data for optimal performance
DataContext.get_current().enable_progress_bars = False

# Initialize Ray with error handling
try:
    if not ray.is_initialized():
        ray.init(
            ignore_reinit_error=True,
            logging_level=logging.INFO,
            include_dashboard=True
        )
    logger.info("Ray cluster initialized successfully")
except Exception as e:
    logger.error(f"Failed to initialize Ray: {e}")
    raise

# Display cluster information
print("="*60)
print("RAY DATA ETL TEMPLATE - ENVIRONMENT STATUS")
print("="*60)
print(f"Ray version: {ray.__version__}")
print(f"Python version: {sys.version}")
print(f"Cluster resources: {ray.cluster_resources()}")
print(f"Dashboard URL: {ray.get_dashboard_url()}")
print("="*60)


2025-08-28 00:38:22,820	INFO worker.py:1771 -- Connecting to existing Ray cluster at address: 10.0.248.128:6379...
2025-08-28 00:38:22,832	INFO worker.py:1942 -- Connected to Ray cluster. View the dashboard at https://session-g3cs331q2c6ppx12j9ir5htn6l.i.anyscaleuserdata.com 
2025-08-28 00:38:22,835	INFO packaging.py:380 -- Pushing file package 'gcs://_ray_pkg_08c553810358104c2d14814f4a773e46518ed2d6.zip' (0.11MiB) to Ray cluster...
2025-08-28 00:38:22,836	INFO packaging.py:393 -- Successfully pushed file package 'gcs://_ray_pkg_08c553810358104c2d14814f4a773e46518ed2d6.zip'.


Ray version: 2.49.0
Ray cluster resources: {'anyscale/node-group:8CPU-32GB': 10.0, 'anyscale/provider:aws': 11.0, 'CPU': 88.0, 'object_store_memory': 105541541064.0, 'memory': 377957122048.0, 'anyscale/cpu_only:true': 11.0, 'node:10.0.246.59': 1.0, 'anyscale/region:us-west-2': 11.0, 'node:10.0.228.88': 1.0, 'node:10.0.195.252': 1.0, 'anyscale/node-group:head': 1.0, 'node:__internal_head__': 1.0, 'node:10.0.248.128': 1.0, 'node:10.0.222.37': 1.0, 'node:10.0.228.223': 1.0, 'node:10.0.234.204': 1.0, 'node:10.0.218.75': 1.0, 'node:10.0.198.227': 1.0, 'node:10.0.208.38': 1.0, 'node:10.0.238.242': 1.0}


## Quick Start: Your First ETL Pipeline

Let's build a complete ETL pipeline in just 5 minutes to see Ray Data in action!

### Step 1: Create Sample Data

We'll start with a simple dataset to demonstrate core ETL concepts:


In [ ]:
# Create sample customer data
def generate_sample_customers(num_customers: int = 1000) -> List[Dict[str, Any]]:
    """Generate realistic sample customer data for ETL demonstration."""
    np.random.seed(42)  # For reproducible results
    
    customers = []
    for i in range(num_customers):
        customer = {
            'customer_id': f"CUST_{i:06d}",
            'name': f"Customer {i}",
            'email': f"customer{i}@example.com",
            'age': np.random.randint(18, 80),
            'income': np.random.normal(50000, 20000),
            'city': np.random.choice(['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix']),
            'signup_date': datetime.now() - timedelta(days=np.random.randint(0, 365)),
            'is_premium': np.random.choice([True, False], p=[0.3, 0.7])
        }
        customers.append(customer)
    
    return customers

# Generate and create Ray dataset
sample_customers = generate_sample_customers(1000)
customers_ds = ray.data.from_items(sample_customers)

print(f"Created dataset with {customers_ds.count()} customers")
print(f"Schema: {customers_ds.schema()}")
print("\nSample records:")
customers_ds.take(3)


### Step 2: Transform Data

Now let's apply some business logic transformations:


In [ ]:
# Transform customer data with business logic
def transform_customer_batch(batch: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """Apply business transformations to customer data."""
    import pandas as pd
    
    df = pd.DataFrame(batch)
    
    # Calculate derived fields
    df['income_tier'] = pd.cut(df['income'], 
                              bins=[0, 30000, 60000, 100000, float('inf')], 
                              labels=['Low', 'Medium', 'High', 'Very High'])
    
    df['age_group'] = pd.cut(df['age'], 
                            bins=[0, 25, 35, 50, 65, 100], 
                            labels=['18-25', '26-35', '36-50', '51-65', '65+'])
    
    df['customer_value'] = df['income'] * (1.2 if df['is_premium'] else 1.0)
    df['days_since_signup'] = (datetime.now() - pd.to_datetime(df['signup_date'])).dt.days
    
    return df.to_dict('records')

# Apply transformations
transformed_customers = customers_ds.map_batches(
    transform_customer_batch,
    batch_size=100,
    concurrency=4
)

print("Transformation completed!")
print(f"Transformed dataset schema: {transformed_customers.schema()}")
print("\nSample transformed records:")
transformed_customers.take(2)


### Step 3: Aggregate and Analyze

Let's perform some business intelligence aggregations:


In [ ]:
# Perform business intelligence aggregations
customer_analytics = (
    transformed_customers
    .groupby("city")
    .aggregate(
        Count().alias("customer_count"),
        Mean("income").alias("avg_income"),
        Mean("customer_value").alias("avg_customer_value"),
        Sum("customer_value").alias("total_value")
    )
    .sort("total_value", descending=True)
)

print("Customer Analytics by City:")
print("="*50)
customer_analytics.show(5)

# Calculate overall metrics
total_customers = transformed_customers.count()
premium_customers = transformed_customers.filter(lambda x: x["is_premium"]).count()
avg_income = transformed_customers.aggregate(Mean("income"))

print(f"\nOverall Metrics:")
print(f"Total customers: {total_customers:,}")
print(f"Premium customers: {premium_customers:,} ({premium_customers/total_customers*100:.1f}%)")
print(f"Average income: ${avg_income:.2f}")

print("\nQuick Start Complete! You've built your first Ray Data ETL pipeline.")


## Understanding Ray Data and ETL

### The Challenge: Modern Data Processing Complexity

**The Problem:**
Modern data teams face unprecedented challenges in building scalable ETL pipelines:

- **Scale Requirements**: Processing terabytes to petabytes of data daily
- **Technology Fragmentation**: Multiple tools for different data types and processing needs
- **Python Performance**: Traditional distributed systems (Spark, Dask) have JVM overhead
- **AI Integration**: Need to process both structured business data and unstructured AI content
- **Operational Complexity**: Managing clusters, monitoring performance, handling failures
- **Cost Management**: Balancing performance with infrastructure costs

**Real-World Impact:**
- **Amazon**: Migrated exabyte-scale workloads, saving $120M annually
- **Netflix**: Processes 100x more data for recommendation systems
- **Financial Services**: Handles fraud detection at massive scale with real-time requirements

### The Solution: Ray Data's Unified Platform

**Ray Data** is a distributed data processing library that addresses these challenges:

**Core Capabilities:**
- **Unified Processing**: Handle traditional ETL and AI workloads on the same platform
- **Python-Native Performance**: No JVM overhead - pure Python performance at scale
- **Automatic Scaling**: Seamlessly scale from single machines to thousands of cores
- **Streaming Architecture**: Process datasets larger than cluster memory
- **Production Ready**: Built-in fault tolerance, monitoring, and enterprise features

**Technical Advantages:**
- **Lazy Evaluation**: Optimizes entire pipelines before execution
- **Operator Fusion**: Combines compatible operations for efficiency
- **Memory Management**: Intelligent caching and streaming for large datasets
- **Fault Tolerance**: Automatic recovery from node failures
- **Resource Optimization**: Dynamic resource allocation based on workload

### The Impact: Transformative Results

**Performance Improvements:**
- **5.1x faster** processing with RayTurbo optimizations
- **82% cost reduction** compared to traditional solutions
- **90%+ GPU utilization** for AI workloads
- **Sub-second latency** for real-time processing

**Business Value:**
- **Faster Time-to-Insight**: Reduce data processing time from hours to minutes
- **Cost Optimization**: Lower infrastructure costs through better resource utilization
- **Operational Excellence**: Simplified deployment and monitoring
- **Future-Proof Architecture**: Seamlessly evolve from ETL to AI workloads

**Traditional & Current Workloads:**
- **Business ETL**: Customer analytics, financial reporting, operational dashboards
- **Classical ML**: Recommendation systems, fraud detection, predictive analytics
- **Data Engineering**: Large-scale data cleaning, transformation, and aggregation

**Next-Generation Workloads:**
- **Multimodal AI**: Processing text, images, video, and audio together
- **LLM Pipelines**: Fine-tuning, embedding generation, and batch inference
- **Computer Vision**: Image preprocessing and model inference at scale
- **Compound AI Systems**: Orchestrating multiple models and traditional ML

### Ray Data vs Traditional Tools

Let's understand how Ray Data compares to other data processing tools across traditional and modern workloads:

| Feature | Ray Data | Pandas | Spark | Dask |
|---------|----------|--------|-------|------|
| **Scale** | Multi-machine | Single-machine | Multi-machine | Multi-machine |
| **Memory Strategy** | Streaming | In-memory | Mixed | In-Memory |
| **Python Performance** | Native (no JVM) | Native | JVM overhead | Native |
| **CPU Clusters** | Excellent | Single-node | Excellent | Good |
| **GPU Support** | Native | None | Limited | Limited |
| **Classical ML** | Excellent | Limited | Limited | Good |
| **Multimodal Data** | Optimized | Limited | Limited | Limited |
| **Fault Tolerance** | Built-in | None | Built-in | Limited |

### Real-World Impact Across All Workloads

Organizations worldwide are seeing dramatic results with Ray Data for both traditional and advanced workloads:

**Traditional ETL & Analytics:**
- **Amazon**: Migrated an exabyte-scale workload from Spark to Ray Data, cutting costs by **82%** and saving **$120 million annually**
- **Instacart**: Processing **100x more data** for recommendation systems and business analytics
- **Financial Services**: Major banks using Ray Data for fraud detection and risk analytics at scale

**Modern AI & ML:**
- **Niantic**: Reduced code complexity by **85%** while scaling AR/VR data pipelines
- **Canva**: Cut cloud costs in **half** while processing design assets and user data
- **Pinterest**: Boosted GPU utilization to **90%+** for image processing and recommendations

Ray Data provides a unified platform that excels at traditional ETL, classical ML, and next-generation AI workloads - eliminating the need for multiple specialized systems.


## Ray Data Architecture & Concepts

### Step 1: Understanding Ray Data's Architecture

Ray Data's architecture is designed for the modern era of computing, addressing the unique challenges of AI and data processing:

**Key Architectural Principles:**
- **Python-Native**: No JVM overhead or serialization bottlenecks
- **Heterogeneous Compute**: Seamlessly orchestrates CPUs, GPUs, and other accelerators
- **Dynamic Workloads**: Adapts to varying compute needs in real-time
- **Fault Tolerance**: Handles failures gracefully at massive scale

### Step 2: Core Concepts

**Datasets and Blocks:**
A **Dataset** in Ray Data is a distributed collection of data divided into **blocks** - chunks that can be processed independently.

**Block Characteristics:**
- Each block contains 1-128 MB of data
- Stored in Ray's distributed object store
- Operations applied in parallel across the cluster
- Block size affects performance (too small = overhead, too large = memory issues)

**Execution Model:**
Ray Data uses **lazy execution** by default - operations build a plan first, then execute for optimization.

In [2]:
# Let's create a simple dataset to understand blocks
# Create sample data
import numpy as np

data = [{"id": i, "name": f"name_{i}", "value": float(np.random.rand())} for i in range(1000)]
ds = ray.data.from_items(data)

print(f"Dataset: {ds}")
print(f"Number of blocks: {ds.num_blocks()}")
print(f"Schema: {ds.schema()}")

print("\nFirst 3 rows:")
for i, row in enumerate(ds.take(3)):
    print(f"Row {i}: {row}")


2025-08-28 00:38:23,206	INFO dataset.py:3248 -- Tip: Use `take_batch()` instead of `take() / show()` to return records in pandas or numpy batch format.
2025-08-28 00:38:23,208	INFO logging.py:295 -- Registered dataset logger for dataset dataset_327_0
2025-08-28 00:38:23,219	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_327_0. Full logs are in /tmp/ray/session_2025-08-27_22-21-09_261930_2324/logs/ray-data
2025-08-28 00:38:23,220	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_327_0: InputDataBuffer[Input] -> LimitOperator[limit=3]
2025-08-28 00:38:23,222	WARNING resource_manager.py:134 -- ⚠️  Ray's object store is configured to use only 27.9% of available memory (98.3GiB out of 352.0GiB total). For optimal Ray Data performance, we recommend setting the object store to at least 50% of available memory. You can do this by setting the 'object_store_memory' parameter when calling ray.init() or by setting the RAY_DEFAULT_OBJECT_STORE_MEMORY_PROPO

Dataset: MaterializedDataset(
   num_blocks=200,
   num_rows=1000,
   schema={id: int64, name: string, value: double}
)
Number of blocks: 200
Schema: Column  Type
------  ----
id      int64
name    string
value   double

First 3 rows:


2025-08-28 00:38:24,741	INFO streaming_executor.py:279 -- ✔️  Dataset dataset_327_0 execution finished in 1.52 seconds


Row 0: {'id': 0, 'name': 'name_0', 'value': 0.10075406674150811}
Row 1: {'id': 1, 'name': 'name_1', 'value': 0.05100088950394521}
Row 2: {'id': 2, 'name': 'name_2', 'value': 0.467035054122913}


#### 2. Execution Model

Ray Data uses **lazy execution** by default, meaning operations are not executed immediately but are planned and optimized before execution.

**Lazy Execution Benefits:**
- **Optimization**: Ray Data can optimize the entire pipeline before execution
- **Memory efficiency**: Only necessary data is loaded into memory
- **Fault tolerance**: Can restart from intermediate points if failures occur

<div class="alert alert-block alert-warning">
<b> Understanding Execution:</b><br>
<b>Lazy:</b> Build a plan first, then execute (default)<br>
<b>Eager:</b> Execute operations immediately as they're called<br><br>
Lazy execution allows Ray Data to optimize your entire pipeline for better performance!
</div>

Ray Data also utilizes **streaming execution**, which helps combine the best of structured streaming and batch inference.  Streaming execution doesn’t wait for one operator to complete to start the next. Each operator takes in and outputs a stream of blocks. This approach allows you to process datasets that are too large to fit in your cluster’s memory and can be 

![Streaming Execution](https://images.ctfassets.net/xjan103pcp94/6EpeVQ743xNDZZfZyEfKFr/16069060673a8667b08be34aa828cc60/image8.jpg)



In [3]:
import pandas as pd
import numpy as np

def transform(batch: pd.DataFrame) -> pd.DataFrame:
    batch = batch.copy()
    batch["value"] = batch["value"] * 2
    batch = batch[batch["value"] > 1.0]
    batch["category"] = np.where(batch["value"] > 1.5, "high", "medium")
    return batch

ds_lazy = ds.map_batches(transform, batch_format="pandas")  # still lazy

# Calling take() to actually execute 
result = ds_lazy.take(5)


2025-08-28 00:38:24,844	INFO logging.py:295 -- Registered dataset logger for dataset dataset_329_0
2025-08-28 00:38:24,848	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_329_0. Full logs are in /tmp/ray/session_2025-08-27_22-21-09_261930_2324/logs/ray-data
2025-08-28 00:38:24,848	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_329_0: InputDataBuffer[Input] -> TaskPoolMapOperator[MapBatches(transform)] -> LimitOperator[limit=5]
2025-08-28 00:38:25,332	INFO streaming_executor.py:279 -- ✔️  Dataset dataset_329_0 execution finished in 0.48 seconds


#### 3. Lazy vs Eager Operations

Understanding which operations are lazy vs eager is crucial for performance optimization:

<div class="alert alert-block alert-success">
<b> Lazy Operations (Default):</b><br>
These operations build a computation plan but don't execute immediately. They allow Ray Data to optimize the entire pipeline before execution.
</div>

**Lazy Operations (Build Plan, Execute Later):**
- **Data Reading:** `read_parquet()`, `read_csv()`, `read_json()`, `from_pandas()`
- **Transformations:** `map_batches()`, `map()`, `flat_map()`
- **Filtering:** `filter()`, `select_columns()`, `drop_columns()`
- **Joins:** `join()`, `union()`
- **Aggregations:** `groupby().aggregate()`, `groupby().sum()`, `groupby().mean()`
- **Sorting:** `sort()`, `sort_by()`
- **Sampling:** `sample()`, `random_sample()`
- **Repartitioning:** `repartition()`, `coalesce()`
- **Window Operations:** `rolling()`, `window()`

<div class="alert alert-block alert-warning">
<b> Eager Operations (Execute Immediately):</b><br>
These operations trigger immediate execution and materialize results. Use them strategically to control when computation happens.
</div>

**Eager Operations (Execute Immediately):**
- **Materialization:** `materialize()` - Force execution and cache results
- **Data Access:** `take()`, `take_batch()`, `show()`, `to_pandas()`, `to_arrow()`
- **Counts:** `count()`, `num_rows()`, `num_blocks()`
- **Schema:** `schema()` - Returns schema without materializing data
- **Writing:** `write_parquet()`, `write_csv()`, `write_json()`
- **Iteration:** `iter_batches()`, `iter_rows()`
- **Statistics:** `stats()`, `mean()`, `std()`, `min()`, `max()`

**Best Practices:**
- **Use lazy operations** for building complex pipelines
- **Use eager operations** only when you need immediate results
- **Batch eager operations** to minimize execution overhead
- **Use `materialize()`** to cache expensive intermediate results

**Example: Lazy vs Eager Usage**
```python
# Lazy pipeline (no execution yet)
lazy_pipeline = (
    ds
    .filter(lambda x: x["value"] > 0.5)  # Lazy
    .map_batches(transform)              # Lazy
    .groupby("category")                 # Lazy
    .aggregate(Count(), Mean("value"))   # Lazy
)

# Eager execution (triggers computation)
result = lazy_pipeline.take(10)          # Eager - executes entire pipeline
```


## Part 3: Extract - Reading Data

The **Extract** phase involves reading data from various sources. Ray Data provides built-in connectors for many common data sources and makes it easy to scale data reading across a distributed cluster, especially for the **multimodal data** that powers modern AI applications.

### The Multimodal Data Revolution

Today's AI applications process vastly more complex data than traditional ETL pipelines:

<div class="alert alert-block alert-success">
<b> The Scale of Modern Data:</b>
<ul>
    <li><b>Unstructured Data Growth:</b> Now outpaces structured data by 10x+ in most organizations</li>
    <li><b>Video Processing:</b> Companies like OpenAI (Sora), Pinterest, and Apple process petabytes of multimodal data daily</li>
    <li><b>Foundation Models:</b> Require processing millions of images, videos, and documents</li>
    <li><b>AI-Powered Processing:</b> Every aspect of data processing is becoming AI-enhanced</li>
</ul>
</div>

### How Ray Data Reads Data Under the Hood

When you read data with Ray Data, here's what happens:

1. **File Discovery**: Ray Data discovers all files matching your path pattern
2. **Task Creation**: Files are distributed across Ray tasks (typically one file per task)
3. **Parallel Reading**: Multiple tasks read files simultaneously across the cluster
4. **Block Creation**: Each task creates data blocks stored in Ray's object store
5. **Lazy Planning**: The dataset is created but data isn't loaded until needed

This architecture enables Ray Data to efficiently handle both traditional structured data and modern unstructured formats that power AI applications.

<div class="alert alert-block alert-info">
<b> Built-in Data Sources:</b>
<ul>
    <li><b>Structured:</b> Parquet, CSV, JSON, Arrow</li>
    <li><b>Unstructured:</b> Images, Videos, Audio, Binary files</li>
    <li><b>Databases:</b> MongoDB, MySQL, PostgreSQL, Snowflake</li>
    <li><b>Cloud Storage:</b> S3, GCS, Azure Blob Storage</li>
    <li><b>Data Lakes:</b> Delta Lake, Iceberg (via RayTurbo)</li>
    <li><b>ML Formats:</b> TensorFlow Records, PyTorch datasets</li>
    <li><b>Memory:</b> Python lists, NumPy arrays, Pandas DataFrames</li>
</ul>
</div>

### Enterprise-Grade Data Connectivity

For enterprise environments, **Anyscale** provides additional connectors and optimizations:
- **Enhanced Security**: Integration with enterprise identity systems
- **Governance Controls**: Data lineage and access controls
- **Performance Optimization**: RayTurbo's streaming metadata fetching provides up to **4.5x faster** data loading
- **Hybrid Deployment**: Support for Kubernetes, on-premises, and multi-cloud environments


### Accessing TPC-H Benchmark Data for Our ETL Examples

We'll use the industry-standard TPC-H benchmark dataset. This provides realistic enterprise-scale data that's used by companies worldwide to evaluate data processing systems and represents real business scenarios with complex relationships between customers, orders, suppliers, and products.


In [4]:
# Using TPC-H Benchmark Dataset - Industry Standard for Data Processing
# TPC-H is the gold standard benchmark for decision support systems and analytics

# TPC-H S3 data location
TPCH_S3_PATH = "s3://ray-benchmark-data/tpch/parquet/sf10"

# TPC-H Schema Overview
tpch_tables = {
    "customer": "Customer master data with demographics and market segments",
    "orders": "\tOrder header information with dates, priorities, and status",
    "lineitem": "Detailed line items for each order (largest table ~6B rows)",
    "part": "\tParts catalog with specifications and retail prices", 
    "supplier": "Supplier information including contact details and geography",
    "partsupp": "Part-supplier relationships with costs and availability",
    "nation": "\tNation reference data with geographic regions",
    "region": "\tRegional groupings for geographic analysis"
}

print(f"\n TPC-H Schema (8 Tables):")
for table, description in tpch_tables.items():
    print(f"\t{table.upper()}:\t{description}")



 TPC-H Schema (8 Tables):
	CUSTOMER:	Customer master data with demographics and market segments
	ORDERS:		Order header information with dates, priorities, and status
	LINEITEM:	Detailed line items for each order (largest table ~6B rows)
	PART:		Parts catalog with specifications and retail prices
	SUPPLIER:	Supplier information including contact details and geography
	PARTSUPP:	Part-supplier relationships with costs and availability
	NATION:		Nation reference data with geographic regions
	REGION:		Regional groupings for geographic analysis


### Step 3: Real-World Data Sources

Let's work with actual public datasets that demonstrate enterprise ETL patterns:

**Public Data Sources We'll Use:**
- **TPC-H Benchmark**: Industry-standard benchmark for data warehousing
- **Yahoo Finance**: Real financial data for time series analysis
- **Open Datasets**: Various public datasets for different use cases

### Step 4: Data Quality Validation

Before processing any data, it's crucial to validate data quality:


In [ ]:
# Data Quality Validation Functions
def validate_data_quality(dataset: Dataset, dataset_name: str) -> Dict[str, Any]:
    """Comprehensive data quality validation for any dataset."""
    logger.info(f"Validating data quality for {dataset_name}...")
    
    validation_results = {
        'dataset_name': dataset_name,
        'total_records': 0,
        'null_counts': {},
        'data_types': {},
        'duplicate_count': 0,
        'quality_score': 0.0,
        'issues': []
    }
    
    try:
        # Get basic statistics
        validation_results['total_records'] = dataset.count()
        validation_results['data_types'] = dataset.schema()
        
        # Sample data for detailed analysis
        sample_data = dataset.take(1000)
        if not sample_data:
            validation_results['issues'].append("No data available for validation")
            return validation_results
        
        # Convert to pandas for analysis
        import pandas as pd
        df = pd.DataFrame(sample_data)
        
        # Check for null values
        null_counts = df.isnull().sum()
        validation_results['null_counts'] = null_counts.to_dict()
        
        # Check for duplicates
        validation_results['duplicate_count'] = df.duplicated().sum()
        
        # Calculate quality score (0-100)
        total_cells = len(df) * len(df.columns)
        null_cells = null_counts.sum()
        duplicate_rows = validation_results['duplicate_count']
        
        quality_score = max(0, 100 - (null_cells / total_cells * 50) - (duplicate_rows / len(df) * 50))
        validation_results['quality_score'] = quality_score
        
        # Identify specific issues
        if null_cells > 0:
            validation_results['issues'].append(f"Found {null_cells} null values")
        if duplicate_rows > 0:
            validation_results['issues'].append(f"Found {duplicate_rows} duplicate rows")
        if quality_score < 80:
            validation_results['issues'].append("Data quality score below 80%")
        
        logger.info(f"Data quality validation completed for {dataset_name}")
        logger.info(f"Quality score: {quality_score:.1f}%")
        
    except Exception as e:
        logger.error(f"Data quality validation failed for {dataset_name}: {e}")
        validation_results['issues'].append(f"Validation error: {str(e)}")
    
    return validation_results

def display_validation_results(results: Dict[str, Any]) -> None:
    """Display data quality validation results in a readable format."""
    print(f"\n{'='*60}")
    print(f"DATA QUALITY REPORT: {results['dataset_name']}")
    print(f"{'='*60}")
    print(f"Total Records: {results['total_records']:,}")
    print(f"Quality Score: {results['quality_score']:.1f}%")
    print(f"Duplicate Rows: {results['duplicate_count']:,}")
    
    if results['null_counts']:
        print(f"\nNull Value Counts:")
        for column, null_count in results['null_counts'].items():
            if null_count > 0:
                print(f"  {column}: {null_count:,}")
    
    if results['issues']:
        print(f"\nIssues Found:")
        for issue in results['issues']:
            print(f"  WARNING: {issue}")
    else:
        print(f"\nNo data quality issues detected")
    
    print(f"{'='*60}")

# Validate our sample customer data
customer_validation = validate_data_quality(customers_ds, "Sample Customer Data")
display_validation_results(customer_validation)


### Step 5: Real Financial Data Processing

Let's work with actual financial data to demonstrate time series ETL patterns:


In [ ]:
# Real Financial Data Processing
def generate_financial_data(symbols: List[str], days: int = 365) -> List[Dict[str, Any]]:
    """Generate realistic financial data for ETL demonstration."""
    import yfinance as yf
    from datetime import datetime, timedelta
    
    financial_data = []
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days)
    
    for symbol in symbols:
        try:
            # Download real stock data
            ticker = yf.Ticker(symbol)
            hist = ticker.history(start=start_date, end=end_date)
            
            for date, row in hist.iterrows():
                financial_data.append({
                    'symbol': symbol,
                    'date': date.strftime('%Y-%m-%d'),
                    'open': float(row['Open']),
                    'high': float(row['High']),
                    'low': float(row['Low']),
                    'close': float(row['Close']),
                    'volume': int(row['Volume']),
                    'adj_close': float(row['Close']),  # Simplified for demo
                    'daily_return': 0.0,  # Will calculate below
                    'volatility': 0.0,    # Will calculate below
                    'market_cap': 0.0     # Will calculate below
                })
                
        except Exception as e:
            logger.warning(f"Could not fetch data for {symbol}: {e}")
            # Generate synthetic data as fallback
            base_price = 100.0
            for i in range(days):
                date = (start_date + timedelta(days=i)).strftime('%Y-%m-%d')
                price_change = np.random.normal(0, 0.02)  # 2% daily volatility
                base_price *= (1 + price_change)
                
                financial_data.append({
                    'symbol': symbol,
                    'date': date,
                    'open': base_price * 0.99,
                    'high': base_price * 1.02,
                    'low': base_price * 0.98,
                    'close': base_price,
                    'volume': np.random.randint(1000000, 10000000),
                    'adj_close': base_price,
                    'daily_return': price_change,
                    'volatility': 0.0,
                    'market_cap': 0.0
                })
    
    return financial_data

# Generate financial data for major stocks
stock_symbols = ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'TSLA']
logger.info(f"Generating financial data for {len(stock_symbols)} stocks...")

financial_data = generate_financial_data(stock_symbols, days=90)  # 3 months of data
financial_ds = ray.data.from_items(financial_data)

print(f"Generated financial dataset with {financial_ds.count():,} records")
print(f"Schema: {financial_ds.schema()}")

# Validate financial data quality
financial_validation = validate_data_quality(financial_ds, "Financial Market Data")
display_validation_results(financial_validation)


### Step 6: Schema Documentation and Analysis

Understanding your data schema is crucial for effective ETL processing:


In [ ]:
# Comprehensive Schema Analysis
def analyze_dataset_schema(dataset: Dataset, dataset_name: str) -> Dict[str, Any]:
    """Perform comprehensive schema analysis for a dataset."""
    logger.info(f"Analyzing schema for {dataset_name}...")
    
    schema_info = {
        'dataset_name': dataset_name,
        'total_columns': 0,
        'column_details': {},
        'data_types': {},
        'memory_usage': 0,
        'recommendations': []
    }
    
    try:
        # Get schema information
        schema = dataset.schema()
        schema_info['data_types'] = schema
        schema_info['total_columns'] = len(schema)
        
        # Sample data for detailed analysis
        sample_data = dataset.take(1000)
        if sample_data:
            import pandas as pd
            df = pd.DataFrame(sample_data)
            
            # Analyze each column
            for column in df.columns:
                col_info = {
                    'dtype': str(df[column].dtype),
                    'null_count': int(df[column].isnull().sum()),
                    'unique_count': int(df[column].nunique()),
                    'memory_usage': int(df[column].memory_usage(deep=True)),
                    'sample_values': df[column].dropna().head(3).tolist()
                }
                schema_info['column_details'][column] = col_info
            
            # Calculate total memory usage
            schema_info['memory_usage'] = df.memory_usage(deep=True).sum()
            
            # Generate recommendations
            for column, details in schema_info['column_details'].items():
                if details['null_count'] > len(df) * 0.1:  # More than 10% nulls
                    schema_info['recommendations'].append(f"Column '{column}' has {details['null_count']} null values - consider data cleaning")
                
                if details['dtype'] == 'object' and details['unique_count'] < len(df) * 0.1:
                    schema_info['recommendations'].append(f"Column '{column}' has low cardinality - consider categorical encoding")
                
                if 'date' in column.lower() and details['dtype'] == 'object':
                    schema_info['recommendations'].append(f"Column '{column}' appears to be a date but stored as string - consider datetime conversion")
        
        logger.info(f"Schema analysis completed for {dataset_name}")
        
    except Exception as e:
        logger.error(f"Schema analysis failed for {dataset_name}: {e}")
        schema_info['recommendations'].append(f"Analysis error: {str(e)}")
    
    return schema_info

def display_schema_analysis(schema_info: Dict[str, Any]) -> None:
    """Display comprehensive schema analysis results."""
    print(f"\n{'='*70}")
    print(f"SCHEMA ANALYSIS: {schema_info['dataset_name']}")
    print(f"{'='*70}")
    print(f"Total Columns: {schema_info['total_columns']}")
    print(f"Memory Usage: {schema_info['memory_usage'] / 1024:.1f} KB (sample)")
    
    print(f"\nColumn Details:")
    print(f"{'Column Name':<20} {'Data Type':<15} {'Nulls':<8} {'Unique':<8} {'Sample Values'}")
    print(f"{'-'*70}")
    
    for column, details in schema_info['column_details'].items():
        sample_str = str(details['sample_values'][:2]) if details['sample_values'] else 'N/A'
        print(f"{column:<20} {details['dtype']:<15} {details['null_count']:<8} {details['unique_count']:<8} {sample_str}")
    
    if schema_info['recommendations']:
        print(f"\nRecommendations:")
        for i, rec in enumerate(schema_info['recommendations'], 1):
            print(f"  {i}. {rec}")
    
    print(f"{'='*70}")

# Analyze schemas for both datasets
customer_schema = analyze_dataset_schema(customers_ds, "Customer Data")
display_schema_analysis(customer_schema)

financial_schema = analyze_dataset_schema(financial_ds, "Financial Data")
display_schema_analysis(financial_schema)


### Step 7: Advanced Performance Optimization

Let's demonstrate specific optimization techniques for Ray Data ETL pipelines:


In [ ]:
# Advanced Performance Optimization Techniques
def benchmark_operation(dataset: Dataset, operation_name: str, operation_func, *args, **kwargs) -> Dict[str, Any]:
    """Benchmark a Ray Data operation and return performance metrics."""
    import time
    
    start_time = time.time()
    start_memory = ray.cluster_resources().get('memory', 0)
    
    try:
        result = operation_func(*args, **kwargs)
        end_time = time.time()
        end_memory = ray.cluster_resources().get('memory', 0)
        
        # Get result count if possible
        try:
            record_count = result.count() if hasattr(result, 'count') else len(result) if hasattr(result, '__len__') else 0
        except:
            record_count = 0
        
        execution_time = end_time - start_time
        throughput = record_count / execution_time if execution_time > 0 else 0
        
        return {
            'operation': operation_name,
            'execution_time': execution_time,
            'record_count': record_count,
            'throughput': throughput,
            'memory_used': end_memory - start_memory,
            'success': True
        }
        
    except Exception as e:
        end_time = time.time()
        return {
            'operation': operation_name,
            'execution_time': end_time - start_time,
            'record_count': 0,
            'throughput': 0,
            'memory_used': 0,
            'success': False,
            'error': str(e)
        }

def optimize_batch_processing(dataset: Dataset) -> Dict[str, Any]:
    """Demonstrate batch size optimization for different operations."""
    logger.info("Testing batch size optimization...")
    
    batch_sizes = [100, 500, 1000, 2000, 5000]
    results = {}
    
    def test_transform(batch):
        import pandas as pd
        df = pd.DataFrame(batch)
        df['processed'] = df['income'] * 1.1  # Simple transformation
        return df.to_dict('records')
    
    for batch_size in batch_sizes:
        test_ds = dataset.map_batches(test_transform, batch_size=batch_size)
        result = benchmark_operation(dataset, f"map_batches_batch_{batch_size}", lambda: test_ds.take(100))
        results[f"batch_{batch_size}"] = result
    
    return results

def optimize_concurrency(dataset: Dataset) -> Dict[str, Any]:
    """Demonstrate concurrency optimization for different operations."""
    logger.info("Testing concurrency optimization...")
    
    concurrency_levels = [1, 2, 4, 8, 16]
    results = {}
    
    def test_transform(batch):
        import pandas as pd
        df = pd.DataFrame(batch)
        df['processed'] = df['income'] * 1.1
        return df.to_dict('records')
    
    for concurrency in concurrency_levels:
        test_ds = dataset.map_batches(test_transform, concurrency=concurrency)
        result = benchmark_operation(dataset, f"map_batches_concurrency_{concurrency}", lambda: test_ds.take(100))
        results[f"concurrency_{concurrency}"] = result
    
    return results

def demonstrate_operator_fusion(dataset: Dataset) -> Dict[str, Any]:
    """Demonstrate the benefits of operator fusion."""
    logger.info("Demonstrating operator fusion...")
    
    # Non-fused operations (separate steps)
    def non_fused_pipeline():
        step1 = dataset.filter(lambda x: x['income'] > 30000)
        step2 = step1.map_batches(lambda batch: batch)  # Identity transform
        step3 = step2.select_columns(['customer_id', 'income', 'city'])
        return step3.take(100)
    
    # Fused operations (single pipeline)
    def fused_pipeline():
        return (dataset
                .filter(lambda x: x['income'] > 30000)
                .map_batches(lambda batch: batch)
                .select_columns(['customer_id', 'income', 'city'])
                .take(100))
    
    non_fused_result = benchmark_operation(dataset, "Non-Fused Pipeline", non_fused_pipeline)
    fused_result = benchmark_operation(dataset, "Fused Pipeline", fused_pipeline)
    
    return {
        'non_fused': non_fused_result,
        'fused': fused_result,
        'improvement': (non_fused_result['execution_time'] - fused_result['execution_time']) / non_fused_result['execution_time'] * 100 if non_fused_result['execution_time'] > 0 else 0
    }

# Run performance optimization tests
logger.info("Starting performance optimization analysis...")

# Test batch size optimization
batch_results = optimize_batch_processing(customers_ds)
print("\nBatch Size Optimization Results:")
print(f"{'Batch Size':<12} {'Time (s)':<10} {'Throughput':<12} {'Success'}")
print("-" * 50)
for batch_name, result in batch_results.items():
    batch_size = batch_name.split('_')[1]
    print(f"{batch_size:<12} {result['execution_time']:<10.3f} {result['throughput']:<12.0f} {result['success']}")

# Test concurrency optimization
concurrency_results = optimize_concurrency(customers_ds)
print("\nConcurrency Optimization Results:")
print(f"{'Concurrency':<12} {'Time (s)':<10} {'Throughput':<12} {'Success'}")
print("-" * 50)
for conc_name, result in concurrency_results.items():
    conc_level = conc_name.split('_')[1]
    print(f"{conc_level:<12} {result['execution_time']:<10.3f} {result['throughput']:<12.0f} {result['success']}")

# Test operator fusion
fusion_results = demonstrate_operator_fusion(customers_ds)
print(f"\nOperator Fusion Results:")
print(f"Non-Fused Time: {fusion_results['non_fused']['execution_time']:.3f}s")
print(f"Fused Time: {fusion_results['fused']['execution_time']:.3f}s")
print(f"Performance Improvement: {fusion_results['improvement']:.1f}%")


### Step 8: Resource Monitoring and Observability

Effective monitoring is crucial for production ETL pipelines:


In [ ]:
# Comprehensive Resource Monitoring
def monitor_cluster_resources() -> Dict[str, Any]:
    """Monitor current cluster resource utilization."""
    try:
        resources = ray.cluster_resources()
        nodes = ray.nodes()
        
        monitoring_data = {
            'timestamp': datetime.now().isoformat(),
            'total_cpus': resources.get('CPU', 0),
            'total_memory_gb': resources.get('memory', 0) / (1024**3),
            'object_store_memory_gb': resources.get('object_store_memory', 0) / (1024**3),
            'num_nodes': len(nodes),
            'node_details': []
        }
        
        for node in nodes:
            node_info = {
                'node_id': node['NodeID'],
                'alive': node['Alive'],
                'resources': node['Resources'],
                'cpu_usage': node['Resources'].get('CPU', 0),
                'memory_usage': node['Resources'].get('memory', 0) / (1024**3)
            }
            monitoring_data['node_details'].append(node_info)
        
        return monitoring_data
        
    except Exception as e:
        logger.error(f"Resource monitoring failed: {e}")
        return {'error': str(e)}

def display_resource_status(monitoring_data: Dict[str, Any]) -> None:
    """Display current resource utilization status."""
    if 'error' in monitoring_data:
        print(f"ERROR: Resource monitoring error: {monitoring_data['error']}")
        return
    
    print(f"\n{'='*60}")
    print(f"CLUSTER RESOURCE STATUS - {monitoring_data['timestamp']}")
    print(f"{'='*60}")
    print(f"Total Nodes: {monitoring_data['num_nodes']}")
    print(f"Total CPUs: {monitoring_data['total_cpus']:.1f}")
    print(f"Total Memory: {monitoring_data['total_memory_gb']:.1f} GB")
    print(f"Object Store: {monitoring_data['object_store_memory_gb']:.1f} GB")
    
    print(f"\nNode Details:")
    print(f"{'Node ID':<20} {'Status':<8} {'CPUs':<8} {'Memory (GB)':<12}")
    print(f"{'-'*60}")
    
    for node in monitoring_data['node_details']:
        status = "Alive" if node['alive'] else "Dead"
        print(f"{node['node_id'][:18]:<20} {status:<8} {node['cpu_usage']:<8.1f} {node['memory_usage']:<12.1f}")
    
    print(f"{'='*60}")

def create_performance_dashboard_data(dataset: Dataset, operation_name: str) -> Dict[str, Any]:
    """Create data for performance dashboard visualization."""
    import time
    
    start_time = time.time()
    start_resources = monitor_cluster_resources()
    
    # Perform a sample operation
    sample_result = dataset.take(100)
    
    end_time = time.time()
    end_resources = monitor_cluster_resources()
    
    execution_time = end_time - start_time
    memory_delta = end_resources.get('total_memory_gb', 0) - start_resources.get('total_memory_gb', 0)
    
    return {
        'operation': operation_name,
        'execution_time': execution_time,
        'memory_delta_gb': memory_delta,
        'records_processed': len(sample_result),
        'throughput_per_second': len(sample_result) / execution_time if execution_time > 0 else 0,
        'timestamp': datetime.now().isoformat()
    }

def generate_performance_report(operations_data: List[Dict[str, Any]]) -> None:
    """Generate a comprehensive performance report."""
    print(f"\n{'='*70}")
    print(f"PERFORMANCE REPORT")
    print(f"{'='*70}")
    print(f"{'Operation':<25} {'Time (s)':<10} {'Records':<10} {'Throughput/s':<12} {'Memory Δ':<10}")
    print(f"{'-'*70}")
    
    total_time = 0
    total_records = 0
    
    for op_data in operations_data:
        print(f"{op_data['operation']:<25} {op_data['execution_time']:<10.3f} {op_data['records_processed']:<10} {op_data['throughput_per_second']:<12.0f} {op_data['memory_delta_gb']:<10.3f}")
        total_time += op_data['execution_time']
        total_records += op_data['records_processed']
    
    print(f"{'-'*70}")
    print(f"{'TOTAL':<25} {total_time:<10.3f} {total_records:<10} {total_records/total_time if total_time > 0 else 0:<12.0f} {'N/A':<10}")
    print(f"{'='*70}")

# Monitor current cluster resources
current_resources = monitor_cluster_resources()
display_resource_status(current_resources)

# Create performance monitoring data for different operations
performance_data = []

# Test different operations
operations_to_test = [
    ("Filter Operation", lambda: customers_ds.filter(lambda x: x['income'] > 50000)),
    ("Map Batches", lambda: customers_ds.map_batches(lambda batch: batch)),
    ("Group By", lambda: customers_ds.groupby('city').aggregate(Count())),
    ("Sample Data", lambda: customers_ds.take(100))
]

for op_name, op_func in operations_to_test:
    try:
        op_data = create_performance_dashboard_data(customers_ds, op_name)
        performance_data.append(op_data)
    except Exception as e:
        logger.error(f"Performance monitoring failed for {op_name}: {e}")

# Generate performance report
generate_performance_report(performance_data)

# Display Ray dashboard information
dashboard_url = ray.get_dashboard_url()
print(f"\nRay Dashboard: {dashboard_url}")
print(f"Use the dashboard to monitor real-time cluster performance")
print(f"Look for task execution, memory usage, and resource utilization")


In [5]:
# Read TPC-H Customer Master Data (Traditional Structured Data Processing)
customers_ds = ray.data.read_parquet(f"{TPCH_S3_PATH}/customer")

customers_ds = customers_ds.drop_columns(["column8"])
customers_ds = customers_ds.rename_columns([
"c_custkey",
"c_name",
"c_address",
"c_nationkey",
"c_phone",
"c_acctbal",
"c_mktsegment",
"c_comment",
])

print(" TPC-H Customer Master Data (Traditional ETL):")
print(f"Schema: {customers_ds.schema()}")
print(f"Total customers: {customers_ds.count():,}")
print("Sample customer records:")
customers_ds.limit(25).to_pandas()


/home/ray/anaconda3/lib/python3.12/site-packages/ray/data/_internal/datasource/parquet_datasource.py:750: FutureWarning: The default `file_extensions` for `read_parquet` will change from `None` to ['parquet'] after Ray 2.43, and your dataset contains files that don't match the new `file_extensions`. To maintain backwards compatibility, set `file_extensions=None` explicitly.
  warnings.warn(
2025-08-28 00:38:25,879	INFO logging.py:295 -- Registered dataset logger for dataset dataset_332_0
2025-08-28 00:38:25,894	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_332_0. Full logs are in /tmp/ray/session_2025-08-27_22-21-09_261930_2324/logs/ray-data
2025-08-28 00:38:25,895	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_332_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> TaskPoolMapOperator[ReadFiles] -> TaskPoolMapOperator[MapBatches(drop_columns)] -> LimitOperator[limit=1]
(MapBatches(transform) pid=292427) Error calculating size f

 TPC-H Customer Master Data (Traditional ETL):


2025-08-28 00:38:29,861	INFO streaming_executor.py:279 -- ✔️  Dataset dataset_334_0 execution finished in 1.58 seconds
2025-08-28 00:38:29,867	INFO logging.py:295 -- Registered dataset logger for dataset dataset_335_0
2025-08-28 00:38:29,876	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_335_0. Full logs are in /tmp/ray/session_2025-08-27_22-21-09_261930_2324/logs/ray-data
2025-08-28 00:38:29,876	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_335_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> TaskPoolMapOperator[ReadFiles] -> TaskPoolMapOperator[MapBatches(drop_columns)->Project] -> AggregateNumRows[AggregateNumRows]


Schema: Column        Type
------        ----
c_custkey     int64
c_name        string
c_address     string
c_nationkey   int64
c_phone       string
c_acctbal     double
c_mktsegment  string
c_comment     string


2025-08-28 00:38:30,710	INFO streaming_executor.py:279 -- ✔️  Dataset dataset_335_0 execution finished in 0.83 seconds
2025-08-28 00:38:30,719	INFO logging.py:295 -- Registered dataset logger for dataset dataset_336_0
2025-08-28 00:38:30,729	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_336_0. Full logs are in /tmp/ray/session_2025-08-27_22-21-09_261930_2324/logs/ray-data
2025-08-28 00:38:30,729	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_336_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> TaskPoolMapOperator[ReadFiles] -> TaskPoolMapOperator[MapBatches(drop_columns)] -> LimitOperator[limit=25] -> TaskPoolMapOperator[Project]


Total customers: 1,500,000
Sample customer records:


2025-08-28 00:38:32,123	INFO streaming_executor.py:279 -- ✔️  Dataset dataset_336_0 execution finished in 1.39 seconds


,c_custkey,c_name,c_address,c_nationkey,c_phone,c_acctbal,c_mktsegment,c_comment
0,1,Customer#000000001,"IVhzIApeRb ot,c,E",15,25-989-741-2988,711.56,BUILDING,"to the even, regular platelets. regular, ironi..."
1,2,Customer#000000002,"XSTf4,NCwDVaWNe6tEgvwfmRchLXak",13,23-768-687-3665,121.65,AUTOMOBILE,l accounts. blithely ironic theodolites integr...
2,3,Customer#000000003,MG9kdTD2WBHm,1,11-719-748-3364,7498.12,AUTOMOBILE,"deposits eat slyly ironic, even instructions...."
3,4,Customer#000000004,XxVSJsLAGtn,4,14-128-190-5944,2866.83,MACHINERY,"requests. final, regular ideas sleep final accou"
4,5,Customer#000000005,KvpyuHCplrB84WgAiGV6sYpZq7Tj,3,13-750-942-6364,794.47,HOUSEHOLD,n accounts will have to unwind. foxes cajole a...
5,6,Customer#000000006,"sKZz0CsnMD7mp4Xd0YrBvx,LREYKUWAh yVn",20,30-114-968-4951,7638.57,AUTOMOBILE,tions. even deposits boost according to the sl...
6,7,Customer#000000007,TcGe5gaZNgVePxU5kRrvXBfkasDTea,18,28-190-982-9759,9561.95,AUTOMOBILE,"ainst the ironic, express theodolites. express..."
7,8,Customer#000000008,"I0B10bB0AymmC, 0PrRYBCP1yGJ8xcBPmWhl5",17,27-147-574-9335,6819.74,BUILDING,among the slyly regular theodolites kindle bli...
8,9,Customer#000000009,xKiAFTjUsCuxfeleNqefumTrjS,8,18-338-906-3675,8324.07,FURNITURE,r theodolites according to the requests wake t...
9,10,Customer#000000010,6LrEaV6KR6PLVcgl2ArL Q3rqzLzcT1 v2,5,15-741-346-9870,2753.54,HOUSEHOLD,es regular deposits haggle. fur


In [6]:
from ray.data.aggregate import Count, Mean

segment_analysis = customers_ds.groupby("c_mktsegment").aggregate(
    Count(),             # counts rows per segment
    Mean("c_acctbal"),   # average account balance
    ).rename_columns(["c_mktsegment", "customer_count", "avg_account_balance"]
)

print("Customer Market Segment Distribution:")
segment_analysis.show(5)

2025-08-28 00:38:32,246	INFO logging.py:295 -- Registered dataset logger for dataset dataset_338_0
2025-08-28 00:38:32,257	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_338_0. Full logs are in /tmp/ray/session_2025-08-27_22-21-09_261930_2324/logs/ray-data
2025-08-28 00:38:32,258	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_338_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> TaskPoolMapOperator[ReadFiles] -> TaskPoolMapOperator[MapBatches(drop_columns)->Project] -> AllToAllOperator[Aggregate] -> LimitOperator[limit=1]
2025-08-28 00:38:33,199	WARNING streaming_executor_state.py:790 -- Operator produced a RefBundle with a different schema than the previous one. Previous schema: c_mktsegment: string
count(): int64
mean(c_acctbal): double, new schema: None. This may lead to unexpected behavior.
2025-08-28 00:38:33,211	INFO streaming_executor.py:279 -- ✔️  Dataset dataset_338_0 execution finished in 0.95 seconds
2025-08-28 00:38

Customer Market Segment Distribution:


2025-08-28 00:38:34,066	WARNING streaming_executor_state.py:790 -- Operator produced a RefBundle with a different schema than the previous one. Previous schema: c_mktsegment: string
count(): int64
mean(c_acctbal): double, new schema: None. This may lead to unexpected behavior.
2025-08-28 00:38:34,099	WARNING streaming_executor_state.py:790 -- Operator produced a RefBundle with a different schema than the previous one. Previous schema: c_mktsegment: string
customer_count: int64
avg_account_balance: double, new schema: . This may lead to unexpected behavior.
2025-08-28 00:38:34,112	INFO streaming_executor.py:279 -- ✔️  Dataset dataset_340_0 execution finished in 0.88 seconds


{'c_mktsegment': 'AUTOMOBILE', 'customer_count': 300036, 'avg_account_balance': 4496.230541701662}
{'c_mktsegment': 'BUILDING', 'customer_count': 300276, 'avg_account_balance': 4505.869852402457}
{'c_mktsegment': 'FURNITURE', 'customer_count': 299496, 'avg_account_balance': 4500.162798034031}
{'c_mktsegment': 'HOUSEHOLD', 'customer_count': 299751, 'avg_account_balance': 4499.8627405746765}
{'c_mktsegment': 'MACHINERY', 'customer_count': 300441, 'avg_account_balance': 4492.427445488464}


In [7]:
# Geographic Reference Data - Nations Table
print("TPC-H Nations Reference Data:")
print("Loading geographic data for customer demographics...")

nation_ds = ray.data.read_parquet(f"{TPCH_S3_PATH}/nation")

nation_ds = (
    nation_ds
    .select_columns(["column0", "column1", "column2", "column3"])
    .rename_columns(["n_nationkey", "n_name", "n_regionkey", "n_comment"])
)

print(f"Total nations: {nation_ds.count():,}")

print("\n Sample nation records:")
nation_ds.limit(25).to_pandas()


2025-08-28 00:38:34,219	INFO logging.py:295 -- Registered dataset logger for dataset dataset_343_0
2025-08-28 00:38:34,227	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_343_0. Full logs are in /tmp/ray/session_2025-08-27_22-21-09_261930_2324/logs/ray-data
2025-08-28 00:38:34,227	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_343_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> LimitOperator[limit=1] -> TaskPoolMapOperator[ReadFiles]


TPC-H Nations Reference Data:
Loading geographic data for customer demographics...


2025-08-28 00:38:34,541	INFO streaming_executor.py:279 -- ✔️  Dataset dataset_343_0 execution finished in 0.31 seconds
2025-08-28 00:38:34,548	INFO logging.py:295 -- Registered dataset logger for dataset dataset_345_0
2025-08-28 00:38:34,554	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_345_0. Full logs are in /tmp/ray/session_2025-08-27_22-21-09_261930_2324/logs/ray-data
2025-08-28 00:38:34,554	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_345_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> TaskPoolMapOperator[MapBatches(count_rows)]
2025-08-28 00:38:34,811	INFO streaming_executor.py:279 -- ✔️  Dataset dataset_345_0 execution finished in 0.26 seconds
2025-08-28 00:38:34,817	INFO logging.py:295 -- Registered dataset logger for dataset dataset_346_0
2025-08-28 00:38:34,825	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_346_0. Full logs are in /tmp/ray/session_2025-08-27_22-21-09_261930_2324/logs/ray-

Total nations: 25

 Sample nation records:


2025-08-28 00:38:35,119	INFO streaming_executor.py:279 -- ✔️  Dataset dataset_346_0 execution finished in 0.29 seconds


{'n_nationkey': 0, 'n_name': 'ALGERIA', 'n_regionkey': 0, 'n_comment': ' haggle. carefully final deposits detect slyly agai'}
{'n_nationkey': 1, 'n_name': 'ARGENTINA', 'n_regionkey': 1, 'n_comment': 'al foxes promise slyly according to the regular accounts. bold requests alon'}
{'n_nationkey': 2, 'n_name': 'BRAZIL', 'n_regionkey': 1, 'n_comment': 'y alongside of the pending deposits. carefully special packages are about the ironic forges. slyly special '}
{'n_nationkey': 3, 'n_name': 'CANADA', 'n_regionkey': 1, 'n_comment': 'eas hang ironic, silent packages. slyly regular packages are furiously over the tithes. fluffily bold'}
{'n_nationkey': 4, 'n_name': 'EGYPT', 'n_regionkey': 4, 'n_comment': 'y above the carefully unusual theodolites. final dugouts are quickly across the furiously regular d'}


In [8]:
# Customer Demographics by Nation - Join Analysis
print(" Customer Demographics by Nation:")
print("   Joining customer and nation data for geographic analysis...")

from ray.data.aggregate import Count, Mean, Sum

customer_nation_analysis = (customers_ds
    .join(nation_ds, on=("c_nationkey",), right_on=("n_nationkey",), join_type="inner", num_partitions=100)
    .groupby("n_name")
    .aggregate(
        Count(),
        Mean("c_acctbal"),
        Sum("c_acctbal"),
)
.rename_columns(["n_name", "customer_count", "avg_balance", "total_balance"])
)

customer_nation_analysis.sort("customer_count", descending=True).limit(10).to_pandas()


2025-08-28 00:38:35,231	INFO logging.py:295 -- Registered dataset logger for dataset dataset_350_0
2025-08-28 00:38:35,247	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_350_0. Full logs are in /tmp/ray/session_2025-08-27_22-21-09_261930_2324/logs/ray-data
2025-08-28 00:38:35,248	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_350_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> TaskPoolMapOperator[ReadFiles] -> TaskPoolMapOperator[MapBatches(drop_columns)->Project], InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> TaskPoolMapOperator[ReadFiles] -> JoinOperatorWithPolars[Join(num_partitions=100)] -> AllToAllOperator[Aggregate] -> LimitOperator[limit=1]


 Customer Demographics by Nation:
   Joining customer and nation data for geographic analysis...


2025-08-28 00:38:40,128	WARNING streaming_executor_state.py:790 -- Operator produced a RefBundle with a different schema than the previous one. Previous schema: n_name: string
count(): int64
mean(c_acctbal): double
sum(c_acctbal): double, new schema: None. This may lead to unexpected behavior.
2025-08-28 00:38:40,240	INFO streaming_executor.py:279 -- ✔️  Dataset dataset_350_0 execution finished in 4.99 seconds
2025-08-28 00:38:40,305	INFO logging.py:295 -- Registered dataset logger for dataset dataset_353_0
2025-08-28 00:38:40,398	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_353_0. Full logs are in /tmp/ray/session_2025-08-27_22-21-09_261930_2324/logs/ray-data
2025-08-28 00:38:40,399	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_353_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> TaskPoolMapOperator[ReadFiles] -> TaskPoolMapOperator[MapBatches(drop_columns)->Project], InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles

{'n_name': 'ETHIOPIA', 'customer_count': 60471, 'avg_balance': 4512.16504423608, 'total_balance': 272855132.39}
{'n_name': 'UNITED KINGDOM', 'customer_count': 60381, 'avg_balance': 4482.881766946556, 'total_balance': 270680883.97}
{'n_name': 'FRANCE', 'customer_count': 60316, 'avg_balance': 4483.745413488959, 'total_balance': 270441588.36}
{'n_name': 'INDONESIA', 'customer_count': 60236, 'avg_balance': 4506.17889501295, 'total_balance': 271434191.9200001}
{'n_name': 'INDIA', 'customer_count': 60215, 'avg_balance': 4505.563116167067, 'total_balance': 271302483.03999996}
{'n_name': 'GERMANY', 'customer_count': 60153, 'avg_balance': 4488.417533456352, 'total_balance': 269991779.89}
{'n_name': 'IRAN', 'customer_count': 60101, 'avg_balance': 4512.225304737025, 'total_balance': 271189253.03999996}
{'n_name': 'CHINA', 'customer_count': 60065, 'avg_balance': 4480.9787298759675, 'total_balance': 269149987.40999997}
{'n_name': 'RUSSIA', 'customer_count': 60065, 'avg_balance': 4507.959750603513, 

### Understanding Join Partitioning in Ray Data

The `num_partitions` parameter in joins is crucial for performance optimization. Here's how to choose the right number:

<div class="alert alert-block alert-info">
<b> Join Partitioning Guidelines:</b>
<ul>
    <li><b>Default:</b> Ray Data automatically chooses partitions based on cluster size and data volume</li>
    <li><b>Rule of Thumb:</b> Start with 1-2 partitions per CPU core in your cluster</li>
    <li><b>Memory Consideration:</b> Each partition should fit comfortably in memory</li>
    <li><b>Data Skew:</b> More partitions help with skewed data distribution</li>
    <li><b>Network Overhead:</b> Too many partitions can cause excessive network communication</li>
</ul>
</div>

**Choosing the Right Number of Partitions:**

| Cluster Size | Data Volume | Recommended Partitions | Reasoning |
|--------------|-------------|------------------------|-----------|
| 4-8 cores | < 1GB | 4-8 | One partition per core |
| 16-32 cores | 1-10GB | 16-32 | Balanced parallelism |
| 64+ cores | 10GB+ | 64-128 | Avoid too many small partitions |
| High skew | Any size | 2-4x cores | Handle data imbalance |

**Performance Impact:**
- **Too Few Partitions:** Underutilized cluster, slower processing
- **Too Many Partitions:** Network overhead, task scheduling overhead
- **Optimal Range:** 1-2 partitions per CPU core typically works well

**Monitoring Join Performance:**
```python
# Check join execution plan
print("Join execution plan:")
print(customer_nation_analysis._plan)

# Monitor resource utilization during joins
# Look for balanced task distribution in Ray dashboard
```


In [9]:
# Read TPC-H High-Volume Transactional Data (Orders + Line Items)

# Read Orders table (header information)
orders_ds = ray.data.read_parquet(f"{TPCH_S3_PATH}/orders")

orders_ds = (orders_ds
    .select_columns([f"column{i}" for i in range(9)])
    .rename_columns([
        "o_orderkey",
        "o_custkey",
        "o_orderstatus",
        "o_totalprice",
        "o_orderdate",
        "o_orderpriority",
        "o_clerk",
        "o_shippriority",
        "o_comment",
    ])
)

print("TPC-H Orders Data (Enterprise Transaction Processing):")
print("Sample order records:")
orders_ds.limit(10).to_pandas()

2025-08-28 00:38:50,461	INFO logging.py:295 -- Registered dataset logger for dataset dataset_356_0
2025-08-28 00:38:50,469	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_356_0. Full logs are in /tmp/ray/session_2025-08-27_22-21-09_261930_2324/logs/ray-data
2025-08-28 00:38:50,470	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_356_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> LimitOperator[limit=1] -> TaskPoolMapOperator[ReadFiles]
2025-08-28 00:38:51,744	INFO streaming_executor.py:279 -- ✔️  Dataset dataset_356_0 execution finished in 1.27 seconds
2025-08-28 00:38:51,752	INFO logging.py:295 -- Registered dataset logger for dataset dataset_359_0
2025-08-28 00:38:51,759	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_359_0. Full logs are in /tmp/ray/session_2025-08-27_22-21-09_261930_2324/logs/ray-data
2025-08-28 00:38:51,760	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_359_0: In

 TPC-H Orders Data (Enterprise Transaction Processing):

 Sample order records:


2025-08-28 00:38:53,657	INFO streaming_executor.py:279 -- ✔️  Dataset dataset_359_0 execution finished in 1.90 seconds


{'o_orderkey': 1, 'o_custkey': 369001, 'o_orderstatus': 'O', 'o_totalprice': 186600.18, 'o_orderdate': datetime.date(1996, 1, 2), 'o_orderpriority': '5-LOW', 'o_clerk': 'Clerk#000009506', 'o_shippriority': 0, 'o_comment': 'nstructions sleep furiously among '}
{'o_orderkey': 2, 'o_custkey': 780017, 'o_orderstatus': 'O', 'o_totalprice': 66219.63, 'o_orderdate': datetime.date(1996, 12, 1), 'o_orderpriority': '1-URGENT', 'o_clerk': 'Clerk#000008792', 'o_shippriority': 0, 'o_comment': ' foxes. pending accounts at the pending, silent asymptot'}
{'o_orderkey': 3, 'o_custkey': 1233140, 'o_orderstatus': 'F', 'o_totalprice': 270741.97, 'o_orderdate': datetime.date(1993, 10, 14), 'o_orderpriority': '5-LOW', 'o_clerk': 'Clerk#000009543', 'o_shippriority': 0, 'o_comment': 'sly final accounts boost. carefully regular ideas cajole carefully. depos'}


In [10]:
# Read Line Items table (detailed transaction data - largest table in TPC-H)
lineitem_ds = ray.data.read_parquet(f"{TPCH_S3_PATH}/lineitem")

lineitem_cols = [f"column{str(i).zfill(2)}" for i in range(16)]
lineitem_ds = (lineitem_ds
    .select_columns(lineitem_cols)
    .rename_columns([
        "l_orderkey",
        "l_partkey",
        "l_suppkey",
        "l_linenumber",
        "l_quantity",
        "l_extendedprice",
        "l_discount",
        "l_tax",
        "l_returnflag",
        "l_linestatus",
        "l_shipdate",
        "l_commitdate",
        "l_receiptdate",
        "l_shipinstruct",
        "l_shipmode",
        "l_comment",
    ])
)
#lineitem_ds = lineitem_ds.limit(10000)

print(f"TPC-H Line Items Data (Detailed Transaction Processing):")
print(f"Schema: {lineitem_ds.schema()}")
print(f"Total line items: {lineitem_ds.count():,}")

print("\n Sample line item records:")
lineitem_ds.limit(25).to_pandas()

2025-08-28 00:38:53,780	INFO logging.py:295 -- Registered dataset logger for dataset dataset_362_0
2025-08-28 00:38:53,788	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_362_0. Full logs are in /tmp/ray/session_2025-08-27_22-21-09_261930_2324/logs/ray-data
2025-08-28 00:38:53,789	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_362_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> LimitOperator[limit=1] -> TaskPoolMapOperator[ReadFiles]


2025-08-28 00:38:55,331	INFO streaming_executor.py:279 -- ✔️  Dataset dataset_362_0 execution finished in 1.54 seconds
2025-08-28 00:38:55,338	INFO logging.py:295 -- Registered dataset logger for dataset dataset_364_0
2025-08-28 00:38:55,345	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_364_0. Full logs are in /tmp/ray/session_2025-08-27_22-21-09_261930_2324/logs/ray-data
2025-08-28 00:38:55,345	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_364_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> LimitOperator[limit=1] -> TaskPoolMapOperator[ReadFiles]



 TPC-H Line Items Data (Detailed Transaction Processing):


2025-08-28 00:38:56,747	INFO streaming_executor.py:279 -- ✔️  Dataset dataset_364_0 execution finished in 1.40 seconds
2025-08-28 00:38:56,753	INFO logging.py:295 -- Registered dataset logger for dataset dataset_365_0
2025-08-28 00:38:56,761	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_365_0. Full logs are in /tmp/ray/session_2025-08-27_22-21-09_261930_2324/logs/ray-data
2025-08-28 00:38:56,762	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_365_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> TaskPoolMapOperator[MapBatches(count_rows)]


    Schema: Column           Type
------           ----
l_orderkey       int64
l_partkey        int64
l_suppkey        int64
l_linenumber     int64
l_quantity       int64
l_extendedprice  double
l_discount       double
l_tax            double
l_returnflag     string
l_linestatus     string
l_shipdate       date32[day]
l_commitdate     date32[day]
l_receiptdate    date32[day]
l_shipinstruct   string
l_shipmode       string
l_comment        string


2025-08-28 00:38:57,257	INFO streaming_executor.py:279 -- ✔️  Dataset dataset_365_0 execution finished in 0.50 seconds
2025-08-28 00:38:57,264	INFO logging.py:295 -- Registered dataset logger for dataset dataset_366_0
2025-08-28 00:38:57,271	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_366_0. Full logs are in /tmp/ray/session_2025-08-27_22-21-09_261930_2324/logs/ray-data
2025-08-28 00:38:57,272	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_366_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> LimitOperator[limit=3] -> TaskPoolMapOperator[ReadFiles]


    Total line items: 59,986,052

 Sample line item records:


2025-08-28 00:38:59,309	INFO streaming_executor.py:279 -- ✔️  Dataset dataset_366_0 execution finished in 2.04 seconds


{'l_orderkey': 6000001, 'l_partkey': 1278570, 'l_suppkey': 53607, 'l_linenumber': 1, 'l_quantity': 45, 'l_extendedprice': 69682.95, 'l_discount': 0.09, 'l_tax': 0.05, 'l_returnflag': 'N', 'l_linestatus': 'O', 'l_shipdate': datetime.date(1998, 10, 20), 'l_commitdate': datetime.date(1998, 9, 10), 'l_receiptdate': datetime.date(1998, 11, 15), 'l_shipinstruct': 'COLLECT COD', 'l_shipmode': 'SHIP', 'l_comment': 'es haggle blithely above the silent ac'}
{'l_orderkey': 6000001, 'l_partkey': 921150, 'l_suppkey': 96178, 'l_linenumber': 2, 'l_quantity': 47, 'l_extendedprice': 55042.17, 'l_discount': 0.08, 'l_tax': 0.02, 'l_returnflag': 'N', 'l_linestatus': 'O', 'l_shipdate': datetime.date(1998, 9, 8), 'l_commitdate': datetime.date(1998, 8, 31), 'l_receiptdate': datetime.date(1998, 9, 15), 'l_shipinstruct': 'COLLECT COD', 'l_shipmode': 'AIR', 'l_comment': 'counts. furio'}
{'l_orderkey': 6000002, 'l_partkey': 1832878, 'l_suppkey': 7933, 'l_linenumber': 1, 'l_quantity': 13, 'l_extendedprice': 23540

## Part 4: Transform - Processing Data

The **Transform** phase is where the real data processing happens. Ray Data provides several transformation operations that can be applied to datasets, and understanding how they work under the hood is key to building efficient ETL pipelines that power modern AI applications.

### Transformations for the AI Era

Modern AI workloads require more than traditional data transformations. Ray Data is designed for the era of **compound AI systems** and **agentic workflows** where:

<div class="alert alert-block alert-success">
<b> AI-Powered Transformations:</b>
<ul>
    <li><b>Multimodal Processing:</b> Simultaneously process text, images, video, and audio</li>
    <li><b>Model Inference:</b> Embed ML models directly into transformation pipelines</li>
    <li><b>GPU Acceleration:</b> Seamlessly utilize both CPU and GPU resources</li>
    <li><b>Compound AI:</b> Orchestrate multiple models and traditional ML within single workflows</li>
    <li><b>AI-Enhanced ETL:</b> Use AI to optimize every aspect of data processing</li>
</ul>
</div>

### How Ray Data Processes Transformations

When you apply transformations with Ray Data:

1. **Task Distribution**: Transformations are distributed across Ray tasks/actors
2. **Block-level Processing**: Each task processes one or more blocks independently  
3. **Streaming Execution**: Blocks flow through the pipeline without waiting for all data
4. **Operator Fusion**: Compatible operations are automatically combined for efficiency
5. **Heterogeneous Compute**: Intelligently schedules CPU and GPU work
6. **Fault Tolerance**: Failed tasks are automatically retried

This architecture enables Ray Data to handle everything from traditional business logic to cutting-edge AI inference within the same pipeline.

<div class="alert alert-block alert-info">
<b> Transformation Categories:</b>
<ul>
    <li><b>Row-wise operations:</b> <code>map()</code> - Transform individual rows</li>
    <li><b>Batch operations:</b> <code>map_batches()</code> - Transform groups of rows (ideal for ML inference)</li>
    <li><b>Filtering:</b> <code>filter()</code> - Remove rows based on conditions</li>
    <li><b>Aggregations:</b> <code>groupby()</code> - Group and aggregate data</li>
    <li><b>Joins:</b> <code>join()</code> - Combine datasets</li>
    <li><b>AI Operations:</b> Embed models for inference, embeddings, and feature extraction</li>
    <li><b>Shuffling:</b> <code>random_shuffle()</code>, <code>sort()</code> - Reorder data</li>
</ul>
</div>

While most other data frameworks support similar map operations through UDFs, with Ray Data, these are treated as first-class supported features. Instead of just arbitrary operations sent to partitions, Ray Data has several key advantages:
- Each task can have task-level concurrency and hardware allocation set instead of just global settings for all UDFs.
- These tasks support PyArrow, pandas, and NumPy format, which provides easy integrations to the rest of the Python ecosystem.
- These tasks support stateful actors, which supports initializing expensive steps like downloading an AI model, only once per replica instead of per-invocation.
- For more advanced use cases, Ray Core can be run inside of the task, supporting nested parallelism algorithms. This is useful for HPC-style applications where we need to have complicated compute tasks on top of big data.


<div class="alert alert-block alert-success">
<b> GPU Optimizations:</b>
<ul>
    <li><b>Nvidia RAPIDS:</b> <code>map_batches()</code> - Pandas ETL operations can be sped up using the Nvidia cuDF library to run the slower sections of ETL logic onto GPUs.</li>
    <li><b>Batch Inference:</b> <code>map_batches()</code> - GPU AI batch inference can be used for unstructured data ingestion or LLM processing, amongst many other use cases</li>
    <li><b>AI Training:</b> - Many data pipelines, such as time series analysis, train many small models over sections of the data. These smaller ML models, such as XGBoost models, can be trained using GPUs for faster performance</li>
</ul>
</div>

In this tutorial, we are going to focus on traditional CPU-based ETL workloads, but there are other templates available for batch inference using GPUs if you are interested in learning further.

### Practical ETL Transformations

Let's implement common ETL transformations using our e-commerce data:

#### 1. Data Enrichment with Business Logic


In [12]:
def traditional_etl_enrichment_tpch(batch):
    """
    Traditional ETL transformations for TPC-H business intelligence and reporting
    This demonstrates classic data warehouse-style transformations on enterprise data
    """
    #df = batch.to_pandas() if hasattr(batch, 'to_pandas') else pd.DataFrame(batch)
    df = batch
    
    # Parse order date and create time dimensions (standard BI practice)
    df['o_orderdate'] = pd.to_datetime(df['o_orderdate'])
    df['order_year'] = df['o_orderdate'].dt.year
    df['order_quarter'] = df['o_orderdate'].dt.quarter
    df['order_month'] = df['o_orderdate'].dt.month
    df['order_day_of_week'] = df['o_orderdate'].dt.dayofweek
    
    # Business day classifications (common in traditional ETL)
    df['is_weekend'] = df['order_day_of_week'].isin([5, 6])
    df['quarter_name'] = 'Q' + df['order_quarter'].astype(str)
    df['month_name'] = df['o_orderdate'].dt.month_name()
    
    # Revenue and profit calculations (standard BI metrics)
    df['revenue_tier'] = pd.cut(
        df['o_totalprice'],
        bins=[0, 50000, 150000, 300000, float('inf')],
        labels=['Small', 'Medium', 'Large', 'Enterprise']
    )
    
    # Order priority business rules (TPC-H specific)
    priority_weights = {
        '1-URGENT': 1.0,
        '2-HIGH': 0.8,
        '3-MEDIUM': 0.6,
        '4-NOT SPECIFIED': 0.4,
        '5-LOW': 0.2
    }
    df['priority_weight'] = df['o_orderpriority'].map(priority_weights).fillna(0.4)
    df['weighted_revenue'] = df['o_totalprice'] * df['priority_weight']
    
    # Order status analysis
    df['is_urgent'] = df['o_orderpriority'].isin(['1-URGENT', '2-HIGH'])
    df['is_large_order'] = df['o_totalprice'] > 200000
    df['requires_expedited_processing'] = df['is_urgent'] | df['is_large_order']
    
    # Date-based business logic
    df['days_to_process'] = (pd.to_datetime(df['o_orderdate']) - pd.Timestamp('1992-01-01')).dt.days
    df['is_peak_season'] = df['order_month'].isin([11, 12])  # Nov-Dec peak
    
    return df

def ml_ready_feature_engineering_tpch(batch):
    """
    Modern ML feature engineering for TPC-H data
    This prepares enterprise data for machine learning models
    """
    #df = batch.to_pandas() if hasattr(batch, 'to_pandas') else pd.DataFrame(batch)
    df = batch
    
    # Temporal features for ML models
    df['days_since_epoch'] = (df['o_orderdate'] - pd.Timestamp('1992-01-01')).dt.days
    df['month_sin'] = np.sin(2 * np.pi * df['order_month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['order_month'] / 12)
    df['quarter_sin'] = np.sin(2 * np.pi * df['order_quarter'] / 4)
    df['quarter_cos'] = np.cos(2 * np.pi * df['order_quarter'] / 4)
    
    # Priority encoding for ML (one-hot style features)
    for priority in ['1-URGENT', '2-HIGH', '3-MEDIUM']:
        df[f'is_priority_{priority.split("-")[0]}'] = (df['o_orderpriority'] == priority).astype(int)
    
    # Revenue-based features (common in ML)
    df['log_total_price'] = np.log1p(df['o_totalprice'])  # Log transformation for ML
    df['revenue_per_priority'] = df['o_totalprice'] * df['priority_weight']
    df['weekend_large_order'] = (df['is_weekend'] & df['is_large_order']).astype(int)
    
    # Time-series features for predictive modeling
    df['year_normalized'] = (df['order_year'] - df['order_year'].min()) / (df['order_year'].max() - df['order_year'].min())
    df['seasonal_revenue_multiplier'] = np.where(df['is_peak_season'], 1.2, 1.0)
    
    # Customer key features (for customer analytics)
    df['customer_id_mod_100'] = df['o_custkey'] % 100  # Simple customer segmentation feature
    
    return df

traditional_enriched = orders_ds.map_batches(
    traditional_etl_enrichment_tpch,
    batch_format="pandas",
    batch_size=1024  # Larger batches for efficiency 
)

print("Traditional ETL Results:")
traditional_enriched.limit(25).to_pandas()


2025-08-28 00:39:20,752	INFO logging.py:295 -- Registered dataset logger for dataset dataset_377_0
2025-08-28 00:39:20,760	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_377_0. Full logs are in /tmp/ray/session_2025-08-27_22-21-09_261930_2324/logs/ray-data
2025-08-28 00:39:20,761	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_377_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> LimitOperator[limit=1000] -> TaskPoolMapOperator[ReadFiles] -> TaskPoolMapOperator[MapBatches(traditional_etl_enrichment_tpch)] -> LimitOperator[limit=25]


Traditional ETL Results:


2025-08-28 00:39:26,971	INFO streaming_executor.py:279 -- ✔️  Dataset dataset_377_0 execution finished in 6.21 seconds


,o_orderkey,o_custkey,o_orderstatus,o_totalprice,o_orderdate,o_orderpriority,o_clerk,o_shippriority,o_comment,order_year,...,quarter_name,month_name,revenue_tier,priority_weight,weighted_revenue,is_urgent,is_large_order,requires_expedited_processing,days_to_process,is_peak_season
0,4423681,325555,O,59347.44,1996-12-08,1-URGENT,Clerk#000000386,0,ng the accounts. slyly express pinto beans sleep,1996,...,Q4,December,Medium,1.0,59347.440,True,False,True,1803,True
1,4423682,1075774,F,48526.32,1993-06-03,2-HIGH,Clerk#000006158,0,deposits. special requests integrate blithely...,1993,...,Q2,June,Small,0.8,38821.056,True,False,True,519,False
2,4423683,1015111,F,211218.00,1994-09-03,1-URGENT,Clerk#000008441,0,ackages. carefully express pains boost. bold d...,1994,...,Q3,September,Large,1.0,211218.000,True,True,True,976,False
3,4423684,1448251,F,73936.26,1994-06-24,5-LOW,Clerk#000007302,0,fully around the unusual accounts. care,1994,...,Q2,June,Medium,0.2,14787.252,False,False,False,905,False
4,4423685,224824,O,60869.04,1995-12-04,4-NOT SPECIFIED,Clerk#000005898,0,pending accounts integrate q,1995,...,Q4,December,Medium,0.4,24347.616,False,False,False,1433,True
5,4423686,94375,O,55521.21,1998-04-11,1-URGENT,Clerk#000006090,0,o beans wake slyly along the even packa,1998,...,Q2,April,Medium,1.0,55521.210,True,False,True,2292,False
6,4423687,637748,P,96743.73,1995-04-13,4-NOT SPECIFIED,Clerk#000003578,0,"he silent, sly packages sleep accordi",1995,...,Q2,April,Medium,0.4,38697.492,False,False,False,1198,False
7,4423712,1115005,P,309802.49,1995-04-16,1-URGENT,Clerk#000009508,0,ounts. furiously bold accou,1995,...,Q2,April,Enterprise,1.0,309802.490,True,True,True,1201,False
8,4423713,364759,F,279348.18,1992-02-08,1-URGENT,Clerk#000005668,0,as. instructions about the quickly ironic foxe,1992,...,Q1,February,Large,1.0,279348.180,True,True,True,38,False
9,4423714,4279,F,122838.22,1993-09-12,5-LOW,Clerk#000009293,0,ss accounts. blithe,1993,...,Q3,September,Medium,0.2,24567.644,False,False,False,620,False


In [13]:
# ML-Ready Feature Engineering (Preparing enterprise data for model training/inference)
print("ML-Ready Feature Engineering (Next-Generation Capabilities):")
print("Adding ML features for predictive analytics on enterprise transaction data...")

enriched_orders = traditional_enriched.map_batches(
    ml_ready_feature_engineering_tpch,
    batch_format="pandas",
    batch_size=10000
)
enriched_orders.limit(25).to_pandas()

2025-08-28 00:39:27,181	INFO logging.py:295 -- Registered dataset logger for dataset dataset_379_0
2025-08-28 00:39:27,191	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_379_0. Full logs are in /tmp/ray/session_2025-08-27_22-21-09_261930_2324/logs/ray-data
2025-08-28 00:39:27,191	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_379_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> LimitOperator[limit=1000] -> TaskPoolMapOperator[ReadFiles] -> TaskPoolMapOperator[MapBatches(traditional_etl_enrichment_tpch)->MapBatches(ml_ready_feature_engineering_tpch)] -> LimitOperator[limit=3]



 ML-Ready Feature Engineering (Next-Generation Capabilities):
   Adding ML features for predictive analytics on enterprise transaction data...

 ML-Ready Data Sample:


2025-08-28 00:39:32,975	INFO streaming_executor.py:279 -- ✔️  Dataset dataset_379_0 execution finished in 5.78 seconds
2025-08-28 00:39:32,987	INFO logging.py:295 -- Registered dataset logger for dataset dataset_380_0
2025-08-28 00:39:32,998	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_380_0. Full logs are in /tmp/ray/session_2025-08-27_22-21-09_261930_2324/logs/ray-data
2025-08-28 00:39:32,998	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_380_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> LimitOperator[limit=1000] -> TaskPoolMapOperator[ReadFiles] -> TaskPoolMapOperator[MapBatches(traditional_etl_enrichment_tpch)->MapBatches(ml_ready_feature_engineering_tpch)] -> LimitOperator[limit=25]


{'o_orderkey': np.int64(24000001), 'o_custkey': np.int64(1494782), 'o_orderstatus': 'O', 'o_totalprice': np.float64(73193.66), 'o_orderdate': Timestamp('1997-03-03 00:00:00'), 'o_orderpriority': '1-URGENT', 'o_clerk': 'Clerk#000008224', 'o_shippriority': np.int64(0), 'o_comment': 'deas. carefully final pinto beans cajole a', 'order_year': np.int32(1997), 'order_quarter': np.int32(1), 'order_month': np.int32(3), 'order_day_of_week': np.int32(0), 'is_weekend': np.False_, 'quarter_name': 'Q1', 'month_name': 'March', 'revenue_tier': 'Medium', 'priority_weight': np.float64(1.0), 'weighted_revenue': np.float64(73193.66), 'is_urgent': np.True_, 'is_large_order': np.False_, 'requires_expedited_processing': np.True_, 'days_to_process': np.int64(1888), 'is_peak_season': np.False_, 'days_since_epoch': np.int64(1888), 'month_sin': np.float64(1.0), 'month_cos': np.float64(6.123233995736766e-17), 'quarter_sin': np.float64(1.0), 'quarter_cos': np.float64(6.123233995736766e-17), 'is_priority_1': np.in

2025-08-28 00:39:38,035	INFO streaming_executor.py:279 -- ✔️  Dataset dataset_380_0 execution finished in 5.04 seconds


,o_orderkey,o_custkey,o_orderstatus,o_totalprice,o_orderdate,o_orderpriority,o_clerk,o_shippriority,o_comment,order_year,...,quarter_cos,is_priority_1,is_priority_2,is_priority_3,log_total_price,revenue_per_priority,weekend_large_order,year_normalized,seasonal_revenue_multiplier,customer_id_mod_100
0,4423681,325555,O,59347.44,1996-12-08,1-URGENT,Clerk#000000386,0,ng the accounts. slyly express pinto beans sleep,1996,...,1.000000e+00,1,0,0,10.991181,59347.440,0,0.666667,1.2,55
1,4423682,1075774,F,48526.32,1993-06-03,2-HIGH,Clerk#000006158,0,deposits. special requests integrate blithely...,1993,...,-1.000000e+00,0,1,0,10.789882,38821.056,0,0.166667,1.0,74
2,4423683,1015111,F,211218.00,1994-09-03,1-URGENT,Clerk#000008441,0,ackages. carefully express pains boost. bold d...,1994,...,-1.836970e-16,1,0,0,12.260651,211218.000,1,0.333333,1.0,11
3,4423684,1448251,F,73936.26,1994-06-24,5-LOW,Clerk#000007302,0,fully around the unusual accounts. care,1994,...,-1.000000e+00,0,0,0,11.210972,14787.252,0,0.333333,1.0,51
4,4423685,224824,O,60869.04,1995-12-04,4-NOT SPECIFIED,Clerk#000005898,0,pending accounts integrate q,1995,...,1.000000e+00,0,0,0,11.016496,24347.616,0,0.500000,1.2,24
5,4423686,94375,O,55521.21,1998-04-11,1-URGENT,Clerk#000006090,0,o beans wake slyly along the even packa,1998,...,-1.000000e+00,1,0,0,10.924538,55521.210,0,1.000000,1.0,75
6,4423687,637748,P,96743.73,1995-04-13,4-NOT SPECIFIED,Clerk#000003578,0,"he silent, sly packages sleep accordi",1995,...,-1.000000e+00,0,0,0,11.479831,38697.492,0,0.500000,1.0,48
7,4423712,1115005,P,309802.49,1995-04-16,1-URGENT,Clerk#000009508,0,ounts. furiously bold accou,1995,...,-1.000000e+00,1,0,0,12.643693,309802.490,1,0.500000,1.0,5
8,4423713,364759,F,279348.18,1992-02-08,1-URGENT,Clerk#000005668,0,as. instructions about the quickly ironic foxe,1992,...,6.123234e-17,1,0,0,12.540218,279348.180,1,0.000000,1.0,59
9,4423714,4279,F,122838.22,1993-09-12,5-LOW,Clerk#000009293,0,ss accounts. blithe,1993,...,-1.836970e-16,0,0,0,11.718632,24567.644,0,0.166667,1.0,79


### Understanding Operator Fusion in Ray Data

Operator fusion is a key optimization technique that combines multiple operations into single tasks, reducing overhead and improving performance.

<div class="alert alert-block alert-success">
<b> What is Operator Fusion?</b><br>
Operator fusion combines compatible operations into single tasks, reducing:
<ul>
    <li><b>Task Scheduling Overhead:</b> Fewer tasks to schedule and manage</li>
    <li><b>Memory Transfers:</b> Intermediate results stay in memory</li>
    <li><b>Network Communication:</b> Less data movement between nodes</li>
    <li><b>Serialization Costs:</b> Fewer serialization/deserialization cycles</li>
</ul>
</div>

**When Operator Fusion Happens:**

Ray Data automatically fuses operators when these conditions are met:

1. **Compatible Resource Requirements:**
   - Same CPU/GPU requirements
   - Compatible memory usage patterns
   - No conflicting resource constraints

2. **Sequential Operations:**
   - Operations that can be chained together
   - No data dependencies that prevent fusion
   - Compatible input/output formats

3. **Compatible Execution Contexts:**
   - Same execution environment
   - Compatible error handling
   - Similar retry policies

**Operations That Fuse Well:**

| Operation Type | Fuses With | Benefits |
|----------------|------------|----------|
| `map_batches()` | `map_batches()`, `filter()` | Reduce task overhead |
| `filter()` | `map_batches()`, `select_columns()` | Memory efficiency |
| `select_columns()` | `map_batches()`, `filter()` | Column pruning |
| `map()` | `map()`, `filter()` | Simple transformations |
| `flat_map()` | `map()`, `filter()` | Data reshaping |

**Operations That Don't Fuse:**

- **All-to-All Operations:** `groupby()`, `join()`, `repartition()`
- **Materialization:** `take()`, `to_pandas()`, `write_*()`
- **Cross-Node Operations:** Operations requiring data shuffling
- **Resource-Intensive Operations:** GPU operations with CPU operations

**Monitoring Operator Fusion:**

```python
# Check if operations are fused
ds = ray.data.from_items([{"id": i, "value": i*2} for i in range(1000)])

# Build a pipeline
pipeline = (
    ds
    .map_batches(lambda batch: batch)  # Should fuse
    .filter(lambda x: x["value"] > 100)  # Should fuse
    .map_batches(lambda batch: batch)  # Should fuse
)

# Check execution plan
print("Execution plan:")
print(pipeline._plan)

# Look for fused operators in the plan
# Fused operations appear as single operators
```

**Forcing Fusion vs Preventing Fusion:**

```python
# Ray Data automatically fuses when beneficial
# You generally don't need to manually control fusion

# However, you can influence fusion through:
# 1. Batch size consistency
pipeline = (
    ds
    .map_batches(func1, batch_size=1000)  # Consistent batch size
    .map_batches(func2, batch_size=1000)  # Helps fusion
)

# 2. Resource requirements
pipeline = (
    ds
    .map_batches(cpu_func, num_cpus=1)    # CPU operation
    .map_batches(cpu_func2, num_cpus=1)   # Fuses with CPU operation
)

# 3. Memory usage patterns
pipeline = (
    ds
    .map_batches(low_memory_func)         # Low memory
    .map_batches(low_memory_func2)        # Fuses with low memory
)
```

**Performance Impact of Fusion:**

| Scenario | Without Fusion | With Fusion | Improvement |
|----------|----------------|-------------|-------------|
| **Simple Pipeline** | 3 tasks | 1 task | 3x fewer tasks |
| **Memory Transfer** | 2x data movement | 1x data movement | 2x less I/O |
| **Scheduling Overhead** | High | Low | Significant reduction |
| **End-to-End Latency** | Higher | Lower | 20-50% improvement |

**Best Practices for Fusion:**

1. **Design for Fusion:**
   - Use consistent batch sizes
   - Keep resource requirements similar
   - Avoid unnecessary materialization

2. **Monitor Fusion:**
   - Check execution plans
   - Use Ray dashboard to monitor task distribution
   - Profile performance with and without fusion

3. **Optimize for Fusion:**
   - Group compatible operations together
   - Avoid breaking fusion with materialization
   - Use appropriate batch sizes

**Example: Fusion in Action**

```python
# This pipeline will have multiple fused operations
optimized_pipeline = (
    ray.data.read_parquet("data.parquet")
    .filter(lambda x: x["value"] > 0)           # Fuses with next operation
    .map_batches(transform_batch)               # Fuses with previous and next
    .select_columns(["id", "value", "category"]) # Fuses with previous
    .map_batches(add_derived_fields)            # Fuses with previous
    .filter(lambda x: x["category"] == "A")     # Fuses with previous
)

# Check the execution plan to see fusion
print("Fused execution plan:")
print(optimized_pipeline._plan)
```

**Troubleshooting Fusion Issues:**

If operations aren't fusing as expected:

1. **Check Resource Requirements:**
   ```python
   # Different resource requirements prevent fusion
   ds.map_batches(cpu_func, num_cpus=1)      # CPU
   ds.map_batches(gpu_func, num_gpus=1)      # GPU - won't fuse
   ```

2. **Check Batch Sizes:**
   ```python
   # Inconsistent batch sizes can prevent fusion
   ds.map_batches(func1, batch_size=1000)    # 1000
   ds.map_batches(func2, batch_size=500)     # 500 - may not fuse
   ```

3. **Check Data Dependencies:**
   ```python
   # Some operations inherently can't fuse
   ds.groupby("category")                    # All-to-all operation
   ds.map_batches(transform)                 # Won't fuse with groupby
   ```

<div class="alert alert-block alert-info">
<b> Fusion Optimization Summary:</b><br>
Ray Data automatically optimizes your pipelines through operator fusion. Focus on writing clean, efficient code and let Ray Data handle the optimization. Monitor execution plans to understand how your operations are being fused and optimized.
</div>


#### 2. Aggregations and Analytics

Aggregations are essential for creating summary statistics and business metrics. Ray Data's `groupby()` operations distribute the computation across the cluster.

<div class="alert alert-block alert-warning">
<b> Under the Hood - GroupBy Operations:</b><br>
When you perform a GroupBy operation, Ray Data:<br>
<ol>
    <li><b>Shuffle Phase:</b> Data is redistributed so all records with the same key end up on the same node</li>
    <li><b>Local Aggregation:</b> Each node performs aggregation on its subset of data</li>
    <li><b>Result Collection:</b> Final aggregated results are collected</li>
</ol>
</div>

Let's make several new datasets by aggregating data together.


In [14]:
from ray.data.aggregate import Count, Mean, Sum, Max

#Executive Summary Dashboard - typical BI metrics on enterprise data
print("Executive Dashboard (Traditional BI on TPC-H):")
executive_summary = (enriched_orders
    .groupby("order_quarter")
    .aggregate(
        Count(),
        Sum("o_totalprice"),
        Mean("o_totalprice"),
        Sum("weighted_revenue"),
        Mean("is_urgent"),
    )
    .rename_columns([
        "order_quarter",
        "total_orders",
        "total_revenue",
        "avg_order_value",
        "weighted_revenue",
        "urgent_order_percentage",
    ])
)

print("Quarterly Business Performance:")
executive_summary.limit(25).to_pandas()

2025-08-28 00:39:38,211	INFO logging.py:295 -- Registered dataset logger for dataset dataset_383_0
2025-08-28 00:39:38,221	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_383_0. Full logs are in /tmp/ray/session_2025-08-27_22-21-09_261930_2324/logs/ray-data
2025-08-28 00:39:38,221	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_383_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> LimitOperator[limit=1000] -> TaskPoolMapOperator[ReadFiles] -> TaskPoolMapOperator[MapBatches(traditional_etl_enrichment_tpch)->MapBatches(ml_ready_feature_engineering_tpch)] -> AllToAllOperator[Aggregate] -> LimitOperator[limit=1]


Executive Dashboard (Traditional BI on TPC-H):


2025-08-28 00:40:01,086	WARNING streaming_executor_state.py:790 -- Operator produced a RefBundle with a different schema than the previous one. Previous schema: order_quarter: int64
count(): int64
sum(o_totalprice): double
mean(o_totalprice): double
sum(weighted_revenue): double
mean(is_urgent): double, new schema: None. This may lead to unexpected behavior.
2025-08-28 00:40:01,123	INFO streaming_executor.py:279 -- ✔️  Dataset dataset_383_0 execution finished in 22.90 seconds
2025-08-28 00:40:01,130	INFO logging.py:295 -- Registered dataset logger for dataset dataset_385_0
2025-08-28 00:40:01,139	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_385_0. Full logs are in /tmp/ray/session_2025-08-27_22-21-09_261930_2324/logs/ray-data
2025-08-28 00:40:01,139	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_385_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> LimitOperator[limit=1000] -> TaskPoolMapOperator[ReadFiles] -> TaskPoolMapOper

Quarterly Business Performance:


2025-08-28 00:40:23,711	WARNING streaming_executor_state.py:790 -- Operator produced a RefBundle with a different schema than the previous one. Previous schema: order_quarter: int64
count(): int64
sum(o_totalprice): double
mean(o_totalprice): double
sum(weighted_revenue): double
mean(is_urgent): double, new schema: None. This may lead to unexpected behavior.
2025-08-28 00:40:23,895	WARNING streaming_executor_state.py:790 -- Operator produced a RefBundle with a different schema than the previous one. Previous schema: order_quarter: int64
total_orders: int64
total_revenue: double
avg_order_value: double
weighted_revenue: double
urgent_order_percentage: double, new schema: PandasBlockSchema(names=[], types=[]). This may lead to unexpected behavior.
2025-08-28 00:40:23,930	INFO streaming_executor.py:279 -- ✔️  Dataset dataset_385_0 execution finished in 22.79 seconds


,order_quarter,total_orders,total_revenue,avg_order_value,weighted_revenue,urgent_order_percentage
0,1,3939930,5.954367e+11,151128.743367,3.573891e+11,0.399966
1,2,3971040,5.999666e+11,151085.499182,3.600278e+11,0.400345
2,3,3648472,5.509344e+11,151004.153324,3.307208e+11,0.400452
3,4,3440558,5.199605e+11,151126.804122,3.120457e+11,0.400078


In [15]:

#Operational Analytics - business process optimization
print("Operational Analytics (Enterprise Process Optimization):")
operational_metrics = (enriched_orders
    .groupby("revenue_tier")
    .aggregate(
        Count(),
        Sum("o_totalprice"),
        Mean("priority_weight"),
        Mean("requires_expedited_processing"),
        Sum("is_peak_season"),
    )
    .rename_columns([
        "revenue_tier",
        "order_volume",
        "total_revenue",
        "avg_priority_weight",
        "expedited_processing_rate",
        "peak_season_orders",
    ])
)

# Uncomment for viewing dataset output
print("Performance by Revenue Tier:")
operational_metrics.limit(25).to_pandas()

2025-08-28 00:40:24,067	INFO logging.py:295 -- Registered dataset logger for dataset dataset_388_0
2025-08-28 00:40:24,082	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_388_0. Full logs are in /tmp/ray/session_2025-08-27_22-21-09_261930_2324/logs/ray-data
2025-08-28 00:40:24,083	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_388_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> LimitOperator[limit=1000] -> TaskPoolMapOperator[ReadFiles] -> TaskPoolMapOperator[MapBatches(traditional_etl_enrichment_tpch)->MapBatches(ml_ready_feature_engineering_tpch)] -> AllToAllOperator[Aggregate] -> LimitOperator[limit=1]


Operational Analytics (Enterprise Process Optimization):


2025-08-28 00:40:45,640	WARNING streaming_executor_state.py:790 -- Operator produced a RefBundle with a different schema than the previous one. Previous schema: revenue_tier: string
count(): int64
sum(o_totalprice): double
mean(priority_weight): double
mean(requires_expedited_processing): double
sum(is_peak_season): int64, new schema: None. This may lead to unexpected behavior.
2025-08-28 00:40:45,689	INFO streaming_executor.py:279 -- ✔️  Dataset dataset_388_0 execution finished in 21.61 seconds
2025-08-28 00:40:45,696	INFO logging.py:295 -- Registered dataset logger for dataset dataset_390_0
2025-08-28 00:40:45,704	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_390_0. Full logs are in /tmp/ray/session_2025-08-27_22-21-09_261930_2324/logs/ray-data
2025-08-28 00:40:45,705	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_390_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> LimitOperator[limit=1000] -> TaskPoolMapOperator[ReadFiles

Performance by Revenue Tier:


2025-08-28 00:41:07,447	WARNING streaming_executor_state.py:790 -- Operator produced a RefBundle with a different schema than the previous one. Previous schema: revenue_tier: string
count(): int64
sum(o_totalprice): double
mean(priority_weight): double
mean(requires_expedited_processing): double
sum(is_peak_season): int64, new schema: None. This may lead to unexpected behavior.
2025-08-28 00:41:07,626	WARNING streaming_executor_state.py:790 -- Operator produced a RefBundle with a different schema than the previous one. Previous schema: revenue_tier: string
order_volume: int64
total_revenue: double
avg_priority_weight: double
expedited_processing_rate: double
peak_season_orders: int64, new schema: PandasBlockSchema(names=[], types=[]). This may lead to unexpected behavior.
2025-08-28 00:41:07,655	INFO streaming_executor.py:279 -- ✔️  Dataset dataset_390_0 execution finished in 21.95 seconds


,revenue_tier,order_volume,total_revenue,avg_priority_weight,expedited_processing_rate,peak_season_orders
0,Enterprise,854969,2.868012e+11,0.600440,1.000000,130174
1,Large,6322157,1.351880e+12,0.600089,0.745465,962126
2,Medium,5706652,5.680665e+11,0.600149,0.400155,867669
3,Small,2116222,5.955084e+10,0.600166,0.400108,322330


In [16]:

#Priority-Based Analysis - enterprise order management
print("\n Priority-Based Analysis (Order Management Insights):")
priority_performance = (enriched_orders
    .groupby("o_orderpriority")
    .aggregate(
        Count(),
        Sum("o_totalprice"),
        Mean("o_totalprice"),
        Mean("is_large_order"),
        Mean("is_weekend"),
    )
    .rename_columns([
        "o_orderpriority",
        "priority_orders",
        "priority_revenue",
        "avg_order_value",
        "large_order_rate",
        "weekend_order_rate",
    ])
)

# Uncomment for viewing dataset output
# print("Performance by Order Priority:")
priority_performance.sort("priority_revenue", descending=True).limit(25).to_pandas()

2025-08-28 00:41:07,751	INFO logging.py:295 -- Registered dataset logger for dataset dataset_393_0
2025-08-28 00:41:07,760	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_393_0. Full logs are in /tmp/ray/session_2025-08-27_22-21-09_261930_2324/logs/ray-data
2025-08-28 00:41:07,761	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_393_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> LimitOperator[limit=1000] -> TaskPoolMapOperator[ReadFiles] -> TaskPoolMapOperator[MapBatches(traditional_etl_enrichment_tpch)->MapBatches(ml_ready_feature_engineering_tpch)] -> AllToAllOperator[Aggregate] -> LimitOperator[limit=1]



 Priority-Based Analysis (Order Management Insights):


2025-08-28 00:41:29,471	WARNING streaming_executor_state.py:790 -- Operator produced a RefBundle with a different schema than the previous one. Previous schema: o_orderpriority: string
count(): int64
sum(o_totalprice): double
mean(o_totalprice): double
mean(is_large_order): double
mean(is_weekend): double, new schema: None. This may lead to unexpected behavior.
2025-08-28 00:41:29,508	INFO streaming_executor.py:279 -- ✔️  Dataset dataset_393_0 execution finished in 21.75 seconds
2025-08-28 00:41:29,514	INFO logging.py:295 -- Registered dataset logger for dataset dataset_396_0
2025-08-28 00:41:29,524	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_396_0. Full logs are in /tmp/ray/session_2025-08-27_22-21-09_261930_2324/logs/ray-data
2025-08-28 00:41:29,524	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_396_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> LimitOperator[limit=1000] -> TaskPoolMapOperator[ReadFiles] -> TaskPoolMapO

,o_orderpriority,priority_orders,priority_revenue,avg_order_value,large_order_rate,weekend_order_rate
0,1-URGENT,3003093,4.537567e+11,151096.441152,0.299924,0.286203
1,2-HIGH,3000061,4.534886e+11,151159.783341,0.300221,0.285980
2,4-NOT SPECIFIED,3000260,4.532757e+11,151078.819384,0.299571,0.286167
3,3-MEDIUM,2998940,4.529252e+11,151028.424989,0.299468,0.286202
4,5-LOW,2997646,4.528520e+11,151069.216228,0.299619,0.285374


In [17]:
#Temporal Business Analysis - time-series insights
print("Temporal Analysis (Time-Series Business Intelligence):")
temporal_intelligence = (enriched_orders
    .groupby("order_year")
    .aggregate(
        Count(),
        Sum("o_totalprice"),
        Mean("o_totalprice"),
        Mean("is_peak_season"),
        Mean("is_large_order"),
    )
    .rename_columns([
        "order_year",
        "yearly_orders",
        "yearly_revenue",
        "avg_order_value",
        "peak_season_rate",
        "large_order_percentage",
    ])
)

# Uncomment for viewing dataset output
print("Year-over-Year Performance:")
temporal_intelligence.sort("order_year").limit(25).to_pandas()

2025-08-28 00:41:51,602	INFO logging.py:295 -- Registered dataset logger for dataset dataset_399_0
2025-08-28 00:41:51,611	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_399_0. Full logs are in /tmp/ray/session_2025-08-27_22-21-09_261930_2324/logs/ray-data
2025-08-28 00:41:51,611	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_399_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> LimitOperator[limit=1000] -> TaskPoolMapOperator[ReadFiles] -> TaskPoolMapOperator[MapBatches(traditional_etl_enrichment_tpch)->MapBatches(ml_ready_feature_engineering_tpch)] -> AllToAllOperator[Aggregate] -> LimitOperator[limit=1]


Temporal Analysis (Time-Series Business Intelligence):


2025-08-28 00:42:12,674	WARNING streaming_executor_state.py:790 -- Operator produced a RefBundle with a different schema than the previous one. Previous schema: order_year: int64
count(): int64
sum(o_totalprice): double
mean(o_totalprice): double
mean(is_peak_season): double
mean(is_large_order): double, new schema: None. This may lead to unexpected behavior.
2025-08-28 00:42:12,710	INFO streaming_executor.py:279 -- ✔️  Dataset dataset_399_0 execution finished in 21.10 seconds
2025-08-28 00:42:12,717	INFO logging.py:295 -- Registered dataset logger for dataset dataset_402_0
2025-08-28 00:42:12,726	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_402_0. Full logs are in /tmp/ray/session_2025-08-27_22-21-09_261930_2324/logs/ray-data
2025-08-28 00:42:12,727	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_402_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> LimitOperator[limit=1000] -> TaskPoolMapOperator[ReadFiles] -> TaskPoolMapOpe

Year-over-Year Performance:


2025-08-28 00:42:34,273	WARNING streaming_executor_state.py:790 -- Operator produced a RefBundle with a different schema than the previous one. Previous schema: order_year: int64
count(): int64
sum(o_totalprice): double
mean(o_totalprice): double
mean(is_peak_season): double
mean(is_large_order): double, new schema: None. This may lead to unexpected behavior.
2025-08-28 00:42:34,415	WARNING streaming_executor_state.py:790 -- Operator produced a RefBundle with a different schema than the previous one. Previous schema: order_year: int64
yearly_orders: int64
yearly_revenue: double
avg_order_value: double
peak_season_rate: double
large_order_percentage: double, new schema: PandasBlockSchema(names=[], types=[]). This may lead to unexpected behavior.
2025-08-28 00:42:34,873	WARNING streaming_executor_state.py:790 -- Operator produced a RefBundle with a different schema than the previous one. Previous schema: PandasBlockSchema(names=['order_year', 'yearly_orders', 'yearly_revenue', 'avg_order

,order_year,yearly_orders,yearly_revenue,avg_order_value,peak_season_rate,large_order_percentage
0,1992,2281205,3.444725e+11,151004.621657,0.167060,0.299643
1,1993,2276638,3.440619e+11,151127.204812,0.167049,0.299833
2,1994,2275919,3.440890e+11,151186.860472,0.167508,0.300342
3,1995,2275575,3.437713e+11,151070.064800,0.166866,0.299505
4,1996,2281938,3.447880e+11,151094.386277,0.166771,0.299821
5,1997,2275511,3.436590e+11,151024.990596,0.166723,0.299462
6,1998,1333214,2.014564e+11,151105.820581,0.000000,0.299689


## Part 5: Load - Writing Data

The **Load** phase involves writing the processed data to destination systems. Ray Data supports writing to various formats and destinations, and understanding how this works helps you optimize for your use case.

### How Ray Data Writes Data

When you write data with Ray Data:

1. **Parallel Writing**: Multiple tasks write data simultaneously across the cluster
2. **Partitioned Output**: Data is written as multiple files (one per block typically)
3. **Format Optimization**: Ray Data optimizes the writing process for each format
4. **Streaming Writes**: Large datasets can be written without loading everything into memory

<div class="alert alert-block alert-info">
<b> Supported Output Formats:</b>
<ul>
    <li><b>Files:</b> Parquet, CSV, JSON</li>
    <li><b>Databases:</b> MongoDB, MySQL, PostgreSQL</li>
    <li><b>Cloud Storage:</b> S3, GCS, Azure Blob Storage</li>
    <li><b>Lakehouse Formats:</b> Delta Lake (coming soon), Iceberg, Hudi</li>
    <li><b>Custom:</b> Implement your own writers using <code>FileBasedDatasource</code></li>
</ul>
</div>


In [18]:
# Create output directories for TPC-H processed data
import os
OUTPUT_PATH = "/mnt/cluster_storage/tpch_etl_output"

os.makedirs(OUTPUT_PATH, exist_ok=True)

print(" Writing TPC-H processed data to various formats...")

# Write enriched TPC-H orders to Parquet (best for large enterprise datasets)
print(" Writing enriched TPC-H orders to Parquet...")
enriched_orders.write_parquet(f"{OUTPUT_PATH}/enriched_orders")
executive_summary.write_parquet(f"{OUTPUT_PATH}/executive_summary")
operational_metrics.write_parquet(f"{OUTPUT_PATH}/operational_metrics")
priority_performance.write_parquet(f"{OUTPUT_PATH}/priority_performance")
temporal_intelligence.write_parquet(f"{OUTPUT_PATH}/temporal_intelligence")

2025-08-28 00:42:35,135	INFO logging.py:295 -- Registered dataset logger for dataset dataset_404_0
2025-08-28 00:42:35,148	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_404_0. Full logs are in /tmp/ray/session_2025-08-27_22-21-09_261930_2324/logs/ray-data
2025-08-28 00:42:35,149	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_404_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> LimitOperator[limit=1000] -> TaskPoolMapOperator[ReadFiles] -> TaskPoolMapOperator[MapBatches(traditional_etl_enrichment_tpch)->MapBatches(ml_ready_feature_engineering_tpch)->Write]
2025-08-28 00:42:35,170	WARNING progress_bar.py:120 -- Truncating long operator name to 100 characters. To disable this behavior, set `ray.data.DataContext.get_current().DEFAULT_ENABLE_PROGRESS_BAR_NAME_TRUNCATION = False`.


 Writing TPC-H processed data to various formats...
 Writing enriched TPC-H orders to Parquet...


2025-08-28 00:43:02,396	INFO streaming_executor.py:279 -- ✔️  Dataset dataset_404_0 execution finished in 27.25 seconds
2025-08-28 00:43:02,448	INFO dataset.py:4871 -- Data sink Parquet finished. 15000000 rows and 8.0GB data written.
2025-08-28 00:43:02,454	INFO logging.py:295 -- Registered dataset logger for dataset dataset_407_0
2025-08-28 00:43:02,474	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_407_0. Full logs are in /tmp/ray/session_2025-08-27_22-21-09_261930_2324/logs/ray-data
2025-08-28 00:43:02,475	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_407_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> LimitOperator[limit=1000] -> TaskPoolMapOperator[ReadFiles] -> TaskPoolMapOperator[MapBatches(traditional_etl_enrichment_tpch)->MapBatches(ml_ready_feature_engineering_tpch)] -> AllToAllOperator[Aggregate] -> TaskPoolMapOperator[Project->Write]
2025-08-28 00:43:23,835	WARNING streaming_executor_state.py:790 -- Operator prod

(autoscaler +10m44s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.


## Performance Monitoring and Best Practices

### Step 1: Monitoring Ray Data Performance

**Key Performance Metrics to Track:**
- **Throughput**: Records processed per second
- **Memory Usage**: Peak and average memory consumption
- **CPU Utilization**: Core usage across the cluster
- **I/O Performance**: Read/write speeds and network usage
- **Task Execution Time**: Individual operation performance

### Step 2: Performance Optimization Techniques

**Batch Size Tuning:**
```python
# Optimal batch sizes for different operations
optimal_batch_sizes = {
    'map_batches': 1000,      # Good for most transformations
    'filter': 10000,          # Can handle larger batches
    'groupby': 5000,          # Balance between memory and parallelism
    'join': 2000              # Smaller batches for complex joins
}
```

**Resource Allocation:**
```python
# Configure resources for different workloads
ray.data.DataContext.get_current().execution_options = {
    'concurrency': 4,         # Number of parallel tasks
    'batch_size': 1000,       # Records per batch
    'num_cpus': 1,           # CPUs per task
    'num_gpus': 0            # GPUs per task (if needed)
}
```

### Step 3: Production Best Practices

**Error Handling:**
- Implement comprehensive try-catch blocks
- Use retry mechanisms for transient failures
- Log errors with sufficient context for debugging

**Resource Management:**
- Always call `ray.shutdown()` when done
- Monitor memory usage to prevent OOM errors
- Use streaming operations for large datasets

**Monitoring and Alerting:**
- Set up Ray dashboard monitoring
- Configure alerts for high memory usage
- Track processing times and throughput


## Troubleshooting and Production Considerations

### Common Issues and Solutions

**Issue 1: Out of Memory Errors**
```python
# Problem: Dataset too large for available memory
# Solution: Use streaming operations
ds = ray.data.read_parquet("large_file.parquet")
# Instead of: result = ds.to_pandas()  # This loads everything into memory
# Use: result = ds.take(1000)  # This streams data
```

**Issue 2: Slow Performance**
```python
# Problem: Operations running slowly
# Solutions:
# 1. Increase concurrency
ds.map_batches(func, concurrency=8)

# 2. Optimize batch size
ds.map_batches(func, batch_size=2000)

# 3. Use native Ray Data operations when possible
ds.filter(lambda x: x["value"] > 100)  # Instead of map_batches for simple filters
```

**Issue 3: Task Failures**
```python
# Problem: Tasks failing due to errors
# Solution: Add error handling
def robust_transform(batch):
    try:
        # Your transformation logic
        return transformed_batch
    except Exception as e:
        logger.error(f"Transform failed: {e}")
        return []  # Return empty batch to continue processing
```

### Production Deployment Checklist

**Pre-Deployment:**
- [ ] **Resource Planning**: Calculate memory and CPU requirements
- [ ] **Error Handling**: Implement comprehensive error handling
- [ ] **Logging**: Set up structured logging with appropriate levels
- [ ] **Monitoring**: Configure Ray dashboard and external monitoring
- [ ] **Testing**: Test with production-scale data volumes

**Deployment:**
- [ ] **Environment Variables**: Use environment variables for configuration
- [ ] **Secrets Management**: Secure handling of database credentials and API keys
- [ ] **Resource Limits**: Set appropriate CPU and memory limits
- [ ] **Health Checks**: Implement health check endpoints
- [ ] **Graceful Shutdown**: Handle shutdown signals properly

**Post-Deployment:**
- [ ] **Performance Monitoring**: Track key metrics continuously
- [ ] **Alert Configuration**: Set up alerts for critical issues
- [ ] **Log Analysis**: Regular analysis of logs for issues
- [ ] **Capacity Planning**: Monitor resource usage trends
- [ ] **Backup Strategy**: Implement data backup and recovery procedures

### Key Takeaways

**What You've Learned:**
1. **Ray Data Fundamentals**: Understanding datasets, blocks, and lazy evaluation
2. **ETL Pipeline Building**: Complete extract, transform, load workflows
3. **Performance Optimization**: Tuning batch sizes, concurrency, and resources
4. **Production Readiness**: Error handling, monitoring, and deployment best practices
5. **Real-World Applications**: Processing enterprise-scale data efficiently

**Next Steps:**
1. **Explore Advanced Features**: Window functions, streaming data, GPU acceleration
2. **Integrate with AI Workloads**: Multimodal data processing, model training pipelines
3. **Deploy to Production**: Use Anyscale platform for enterprise deployments
4. **Join the Community**: Connect with other Ray Data users and contributors

**Resources:**
- **Ray Documentation**: https://docs.ray.io/en/latest/data/data.html
- **Ray Data Examples**: https://github.com/ray-project/ray/tree/master/python/ray/data/examples
- **Community Support**: https://discuss.ray.io/
- **Anyscale Platform**: https://www.anyscale.com/

### Action Items

**Immediate Actions:**
1. **Experiment**: Try different batch sizes and concurrency settings
2. **Profile**: Use Ray dashboard to monitor your pipeline performance
3. **Scale**: Test with larger datasets to understand scaling behavior
4. **Optimize**: Apply the performance techniques learned in this template

**Long-term Goals:**
1. **Production Deployment**: Deploy a real ETL pipeline using Ray Data
2. **Team Training**: Share knowledge with your data engineering team
3. **Community Contribution**: Contribute examples or improvements back to the community
4. **Advanced Use Cases**: Explore AI/ML integration with Ray Data


In [ ]:
# Cleanup and Resource Management
def cleanup_resources():
    """Properly clean up Ray resources and display final statistics."""
    try:
        # Display final cluster statistics
        print("="*60)
        print("ETL PIPELINE COMPLETION SUMMARY")
        print("="*60)
        
        # Get cluster resources
        resources = ray.cluster_resources()
        print(f"Cluster Resources Used:")
        print(f"  CPU cores: {resources.get('CPU', 0):.1f}")
        print(f"  Memory: {resources.get('memory', 0) / (1024**3):.1f} GB")
        print(f"  Object Store: {resources.get('object_store_memory', 0) / (1024**3):.1f} GB")
        
        # Display Ray dashboard URL
        dashboard_url = ray.get_dashboard_url()
        print(f"\nRay Dashboard: {dashboard_url}")
        
        print("\nETL Pipeline completed successfully!")
        print("Check the Ray dashboard for detailed performance metrics")
        print("Use the troubleshooting guide for production deployment")
        
    except Exception as e:
        logger.error(f"Error during cleanup: {e}")
    finally:
        # Always attempt to shutdown Ray
        try:
            if ray.is_initialized():
                ray.shutdown()
                logger.info("Ray cluster shutdown completed")
        except Exception as e:
            logger.error(f"Error during Ray shutdown: {e}")

# Run cleanup
cleanup_resources()


In [11]:
from ray.data.aggregate import Count, Mean, Sum

# Order priority analysis - Ray Dataset API
order_priority_analysis = (orders_ds
    .groupby("o_orderpriority")
    .aggregate(
        Count(),
        Mean("o_totalprice"),
        Sum("o_totalprice"),
    )
    .rename_columns(["o_orderpriority", "order_count", "avg_total_price", "total_value"])
)

# Time-based order analysis
orders_with_year = orders_ds.map(lambda r: {**r, "order_year": r["o_orderdate"].year})

yearly_revenue = (orders_with_year
    .groupby("order_year")
    .aggregate(
        Count(),
        Sum("o_totalprice"),
        Mean("o_totalprice"),
    )
    .rename_columns(["order_year", "yearly_orders", "yearly_revenue", "avg_order_value"])
)



2025-08-28 00:38:59,398	INFO logging.py:295 -- Registered dataset logger for dataset dataset_369_0
2025-08-28 00:38:59,406	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_369_0. Full logs are in /tmp/ray/session_2025-08-27_22-21-09_261930_2324/logs/ray-data
2025-08-28 00:38:59,407	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_369_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> LimitOperator[limit=1000] -> TaskPoolMapOperator[ReadFiles] -> AllToAllOperator[Aggregate] -> LimitOperator[limit=1]
2025-08-28 00:39:02,044	WARNING streaming_executor_state.py:790 -- Operator produced a RefBundle with a different schema than the previous one. Previous schema: o_orderpriority: string
count(): int64
mean(o_totalprice): double
sum(o_totalprice): double, new schema: None. This may lead to unexpected behavior.
2025-08-28 00:39:02,066	INFO streaming_executor.py:279 -- ✔️  Dataset dataset_369_0 execution finished in 2.66 seconds
2025-08-28 00

### Comprehensive Filter Operations

Filtering is one of the most common operations in ETL pipelines. Here are comprehensive examples of different filtering patterns:

<div class="alert alert-block alert-info">
<b> Filter Performance Tips:</b>
<ul>
    <li><b>Push Down Filters:</b> Apply filters early in the pipeline to reduce data volume</li>
    <li><b>Indexed Columns:</b> Filter on indexed columns when possible for better performance</li>
    <li><b>Batch Processing:</b> Use `map_batches()` for complex filtering logic</li>
    <li><b>Memory Efficiency:</b> Combine multiple filters into single operations when possible</li>
</ul>
</div>

**1. Basic Value Filtering**
```python
# Simple value comparisons
high_value_customers = customers_ds.filter(lambda x: x["c_acctbal"] > 5000)
recent_orders = orders_ds.filter(lambda x: x["o_orderdate"] > "2023-01-01")
urgent_orders = orders_ds.filter(lambda x: x["o_orderpriority"] == "1-URGENT")
```

**2. Range Filtering**
```python
# Numeric ranges
medium_balance = customers_ds.filter(lambda x: 1000 <= x["c_acctbal"] <= 5000)
recent_orders = orders_ds.filter(lambda x: "2023-01-01" <= x["o_orderdate"] <= "2023-12-31")
```

**3. String Pattern Matching**
```python
# String contains/startswith/endswith
building_customers = customers_ds.filter(lambda x: "BUILDING" in x["c_mktsegment"])
phone_area_555 = customers_ds.filter(lambda x: x["c_phone"].startswith("555"))
```

**4. Multiple Condition Filtering**
```python
# Complex boolean logic
premium_customers = customers_ds.filter(
    lambda x: x["c_acctbal"] > 7500 and x["c_mktsegment"] == "AUTOMOBILE"
)

high_priority_recent = orders_ds.filter(
    lambda x: x["o_orderpriority"] in ["1-URGENT", "2-HIGH"] and 
              x["o_orderdate"] > "2023-06-01"
)
```

**5. Null/None Value Filtering**
```python
# Filter out null values
valid_customers = customers_ds.filter(lambda x: x["c_comment"] is not None)
non_empty_orders = orders_ds.filter(lambda x: x["o_comment"] != "")
```

**6. List/Set Membership Filtering**
```python
# Filter by membership in lists
target_segments = ["AUTOMOBILE", "MACHINERY", "FURNITURE"]
target_customers = customers_ds.filter(lambda x: x["c_mktsegment"] in target_segments)

priority_levels = ["1-URGENT", "2-HIGH"]
priority_orders = orders_ds.filter(lambda x: x["o_orderpriority"] in priority_levels)
```

**7. Date/Time Filtering**
```python
# Date range filtering
from datetime import datetime, timedelta

# Last 30 days
recent_cutoff = datetime.now() - timedelta(days=30)
recent_orders = orders_ds.filter(lambda x: x["o_orderdate"] > recent_cutoff)

# Specific year/quarter
q4_orders = orders_ds.filter(lambda x: x["o_orderdate"].quarter == 4)
```

**8. Complex Business Logic Filtering**
```python
# Multi-step business logic
def is_high_value_customer(customer):
    return (
        customer["c_acctbal"] > 5000 and
        customer["c_mktsegment"] in ["AUTOMOBILE", "MACHINERY"] and
        customer["c_comment"] is not None and
        len(customer["c_comment"]) > 10
    )

high_value = customers_ds.filter(is_high_value_customer)
```

**9. Performance-Optimized Batch Filtering**
```python
# For complex filtering logic, use map_batches
def complex_filter_batch(batch):
    import pandas as pd
    df = pd.DataFrame(batch)
    
    # Complex filtering logic
    filtered_df = df[
        (df["c_acctbal"] > 1000) &
        (df["c_mktsegment"].isin(["AUTOMOBILE", "MACHINERY"])) &
        (df["c_phone"].str.contains("555", na=False)) &
        (df["c_comment"].str.len() > 5)
    ]
    
    return filtered_df.to_dict('records')

filtered_customers = customers_ds.map_batches(
    complex_filter_batch,
    batch_format="pandas"
)
```

**10. Filtering with Error Handling**
```python
# Robust filtering with error handling
def safe_filter(record):
    try:
        return (
            record["c_acctbal"] is not None and
            record["c_acctbal"] > 0 and
            record["c_mktsegment"] in ["AUTOMOBILE", "MACHINERY", "FURNITURE"]
        )
    except (KeyError, TypeError, ValueError):
        return False

safe_filtered = customers_ds.filter(safe_filter)
```

**Filter Performance Monitoring:**
```python
# Monitor filter performance
import time

start_time = time.time()
filtered_ds = customers_ds.filter(lambda x: x["c_acctbal"] > 5000)
result = filtered_ds.take(100)
filter_time = time.time() - start_time

print(f"Filtered {len(result)} records in {filter_time:.2f}s")
print(f"Filter selectivity: {len(result) / customers_ds.count():.2%}")
```


## Summary: Your Journey with Ray Data ETL

Congratulations! You've completed a comprehensive journey through Ray Data for ETL. Let's summarize what you've learned and explore how to take your AI data pipelines to production.

<div class="alert alert-block alert-success">
<b> What You've Mastered:</b>
<ul>
    <li><b>Ray Data Fundamentals:</b> Blocks, lazy execution, streaming processing</li>
    <li><b>Extract Phase:</b> Reading from multiple data sources efficiently, including multimodal data</li>
    <li><b>Transform Phase:</b> Distributed data processing and feature engineering</li>
    <li><b>Load Phase:</b> Writing to various destinations with optimization</li>
    <li><b>Production Patterns:</b> Error handling, monitoring, and data quality</li>
    <li><b>Performance Optimization:</b> Understanding bottlenecks and solutions</li>
</ul>
</div>

### When to Use Ray Data

**Ray Data excels across the full spectrum of data workloads:**

**Traditional ETL & Business Intelligence:**
- **High-volume transaction processing** for e-commerce, finance, and operations
- **Business intelligence** and executive reporting at scale
- **Data warehouse** loading and transformation pipelines
- **CPU cluster optimization** with pure Python performance (no JVM overhead)
- **Traditional analytics** that need to scale beyond single-node tools

**Modern ML & AI Workloads:**
- **Feature engineering** for machine learning at scale
- **Batch inference** on foundation models and LLMs
- **Multimodal data processing** (text, images, video, audio)
- **GPU-accelerated pipelines** for AI applications
- **Real-time model serving** and inference workloads

**Ray Data's Unified Platform Advantage:**
- **One system** for both traditional ETL and cutting-edge AI
- **Seamless evolution** from CPU-based analytics to GPU-powered AI
- **No migration** required as your data needs grow and change
- **Consistent APIs** whether processing structured business data or unstructured AI content

**Ray is proven at scale:**
- Processing **exabyte-scale** workloads
- **1M+ clusters** orchestrated monthly across the Ray ecosystem
- **$120M annual savings** achieved by leading enterprises
- **Traditional workloads** running alongside **next-generation AI** on the same platform

### From Open Source to Enterprise: Anyscale Platform

While Ray Data open source provides powerful capabilities, **Anyscale** offers a unified AI platform for production deployments:

<div class="alert alert-block alert-info">
<b> Anyscale: The Unified AI Platform</b>
<ul>
    <li><b>RayTurbo Runtime:</b> Up to 5.1x performance improvements over open source</li>
    <li><b>Enterprise Governance:</b> Resource quotas, usage tracking, and advanced observability</li>
    <li><b>AI Anywhere:</b> Deploy on Kubernetes, hybrid cloud, or any infrastructure</li>
    <li><b>LLM Suite:</b> Complete capabilities for embeddings, fine-tuning, and serving</li>
    <li><b>Marketplace Ready:</b> Available on AWS and GCP Marketplaces</li>
</ul>
</div>

### Production Deployment Options

**Getting Started:**
1. **Ray Open Source**: Perfect for development and smaller workloads
2. **Anyscale Platform**: Enterprise features with RayTurbo optimizations
3. **Marketplace Deployment**: One-click setup via AWS or GCP Marketplace

### Key Architectural Insights

Understanding how Ray Data works under the hood helps you build better pipelines:

1. **AI-Native Architecture**: Purpose-built for Python, GPUs, and multimodal data
2. **Streaming Execution**: Process datasets larger than cluster memory
3. **Heterogeneous Compute**: Seamlessly orchestrate CPUs, GPUs, and other accelerators
4. **Operator Fusion**: Combines compatible operations for efficiency
5. **Enterprise Scalability**: Proven to scale to 8,000+ nodes

### Production Readiness Checklist

Before deploying Ray Data pipelines to production:

-  **Architecture**: Choose between Ray OSS and Anyscale based on your needs
-  **Performance**: Consider RayTurbo for production workloads requiring maximum efficiency
-  **Governance**: Implement enterprise controls for AI sprawl and cost management
-  **Security**: Leverage enterprise identity integration and access controls
-  **Monitoring**: Use advanced observability tools for optimization insights
-  **Scalability**: Test with realistic data volumes and cluster sizes

### Join the Ray Ecosystem

The Ray community is thriving with **1,000+ contributors** and growing:

1. **Community**: Join the Ray Slack community for support and discussions
2. **Learning**: Access Ray Summit sessions and technical deep-dives
3. **Contributing**: Contribute to the fastest-growing AI infrastructure project
4. **Enterprise Support**: Explore Anyscale for production deployments

<div class="alert alert-block alert-success">
<b> One Platform for All Your Data Workloads</b><br>
You now have the knowledge to build production-ready, scalable data pipelines that handle everything from traditional business ETL to cutting-edge AI applications. Whether you're processing millions of e-commerce transactions for business intelligence or preparing multimodal data for foundation models, Ray Data provides a unified platform that scales with your needs.<br><br>
<b>Start with traditional ETL today, evolve to AI tomorrow - all on the same platform.</b> Ray Data and Anyscale eliminate the complexity of managing multiple systems as your data requirements grow.
</div>

